In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os
import numpy as np
import datasets

os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
import sys
sys.path.append('..')
from utils.reasoning import make_segment, split_cot
from torch.nn.utils.rnn import pad_sequence

In [3]:

# from modeling_rmt.language_modeling import MemoryCell
# from modeling_rmt.experimental import RecurrentWrapperNoSegmentation

In [4]:
# var_holder = Holder()

In [5]:
import torch
import torch.nn as nn
from transformers.modeling_outputs import CausalLMOutputWithCrossAttentions
from transformers.models.gpt2.modeling_gpt2 import GPT2Attention, GPT2MLP

from modeling_rmt.experimental import RecurrentWrapperNoSegmentationGenerate


class RMCrossAttention(torch.nn.Module):
    def __init__(self, config, dropout=False, ff=False):
        super().__init__()
        self.config = config
        if config.hidden_size % config.num_attention_heads != 0 and not hasattr(config, "embedding_size"):
            raise ValueError(
                f"The hidden size ({config.hidden_size}) is not a multiple of the number of attention "
                f"heads ({config.num_attention_heads})"
            )

        hidden_size = config.hidden_size
        self.ln = nn.LayerNorm(hidden_size, eps=config.layer_norm_epsilon)
        self.cross_attn = GPT2Attention(config=config, is_cross_attention=True)
        self.dropout = nn.Dropout(config.attention_dropout) if dropout else None

        inner_dim = config.n_inner if config.n_inner is not None else 4 * hidden_size
        self.ff = GPT2MLP(inner_dim, config) if ff else None

    def forward(self, hidden_states, encoder_hidden_states, attention_mask=None,
                output_attentions=False
                ):
        residual = hidden_states
        hidden_states = self.ln(hidden_states)
        cross_attn_outputs = self.cross_attn(
            hidden_states=hidden_states,
            encoder_hidden_states=encoder_hidden_states,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
        )
        attn_output = cross_attn_outputs[0]
        if self.dropout is not None:
            attn_output = self.dropout(attn_output)
        if self.ff is not None:
            attn_output = self.ff(attn_output)
        hidden_states = residual + attn_output

        outputs = (hidden_states,)
        if output_attentions:
            outputs += (cross_attn_outputs[1:],)

        self.debug_state = dict(encoder_hidden_states=encoder_hidden_states,
                                inp_hidden_states=residual,
                                output=outputs,
                                attn_output=attn_output,
                                cross_attn_outputs=cross_attn_outputs,
        )


        return outputs  # hidden_states, attention_weights


class RMCALayerWrapper(torch.nn.Module):
    def __init__(self, layer, num_mem_tokens, config):
        super().__init__()
        self.layer = layer
        self.create_memory(num_mem_tokens, config.hidden_size)
        self.memory_state = None
        self.mem_read_layer = self.create_mem_read_layer(config)
        self.mem_write_layer = self.create_mem_write_layer(config)
        self.generate_mode = False

    def forward(self, hidden_states, output_attentions=False, *args, **kwargs):
        if self.memory_state is None:
            self.memory_state = self.set_memory(hidden_states)

        mem_read_out = self.mem_read_layer(hidden_states, encoder_hidden_states=self.memory_state,
                                           output_attentions=output_attentions)
        hidden_states, cross_attentions_read = mem_read_out[0], mem_read_out[1:]

        mem_write_out = self.mem_write_layer(self.memory_state, encoder_hidden_states=hidden_states,
                                             output_attentions=output_attentions)
        self.memory_state, cross_attentions_write = mem_write_out[0], mem_write_out[1:]

        out = self.layer(hidden_states, **kwargs, output_attentions=output_attentions)

        self.debug_state = dict(inp_hidden_states=hidden_states, 
                                mem_read_out=mem_read_out,
                                mem_write_out=mem_write_out,
                                memory_state=self.memory_state, output=out,
                                cross_attentions_read=cross_attentions_read,
                                cross_attentions_write=cross_attentions_write)

        return out

    def create_mem_read_layer(self, config):
        mem_read_layer = RMCrossAttention(config, dropout=getattr(self, 'use_dropout', False))
        mem_read_layer.cross_attn.c_proj.weight.data.zero_()
        return mem_read_layer

    def create_mem_write_layer(self, config):
        mem_write_layer = RMCrossAttention(config, dropout=getattr(self, 'use_dropout', False))
        mem_write_layer.cross_attn.c_proj.weight.data.zero_()
        return mem_write_layer

    def create_memory(self, num_mem_tokens, memory_dim):
        self.num_mem_tokens = num_mem_tokens
        memory_weights = torch.randn((num_mem_tokens, memory_dim))
        self.register_parameter('memory', torch.nn.Parameter(memory_weights, requires_grad=True))

    def set_memory(self, hidden_states):
        memory = self.memory.repeat(hidden_states.shape[0], 1, 1)
        return memory

    def reset_memory(self):
        self.memory_state = None

    def detach_memory_state(self):
        if self.memory_state is not None:
            self.memory_state = self.memory_state.detach()


class RMCAMemoryCell(torch.nn.Module):
    def __init__(self, base_model, num_mem_tokens, layers_attr: str = 'transformer.h'):
        super().__init__()
        self.model = base_model
        self.num_mem_tokens = num_mem_tokens

        self.layers = self.model
        self.layers_attrs = layers_attr.split('.')
        for i, attr in enumerate(self.layers_attrs):
            self.layers = getattr(self.layers, attr)

        for i in range(len(self.layers)):
            self.layers[i] = RMCALayerWrapper(
                self.layers[i],
                self.num_mem_tokens,
                self.model.config
            )

    def reset_memory(self):
        for i in range(len(self.layers)):
            self.layers[i].reset_memory()

    def detach_memory_state(self):
        for i in range(len(self.layers)):
            self.layers[i].detach_memory_state()

    def forward(self, *args, **kwargs):
        return self.model(*args, **kwargs)

    def generate(self, input_ids, memory_state, attention_mask=None, **generate_kwargs):
        raise NotImplementedError("Generation is not supported yet.")


class RMCAWrapperNoSegmentationGenerate(RecurrentWrapperNoSegmentationGenerate):
    def forward(self, segments, labels, output_attentions=None, output_hidden_states=None, *args, **kwargs):
        self.memory_cell.reset_memory()

        cell_outputs = []
        for seg_num, segment in enumerate(segments):
            cell_out = self.memory_cell(input_ids=segment['input_ids'],
                                        attention_mask=segment['attention_mask'],
                                        output_attentions=output_attentions,
                                        output_hidden_states=True)
            cell_outputs.append(cell_out)
            self.manage_gradients(seg_num)

        out = self.process_outputs(cell_outputs, segments,
                                   output_attentions=output_attentions,
                                   output_hidden_states=output_hidden_states)
        return out

    def generate(self, segments, **kwargs):
        raise NotImplementedError("Generation is not supported yet.")
        memory_state = None

        for seg_num, segment in enumerate(segments):
            cell_out, memory_state = self.memory_cell(input_ids=segment['input_ids'],
                                                      attention_mask=segment['attention_mask'],
                                                      memory_state=memory_state, output_hidden_states=True)

        generated_segments = []
        for seg_num in range(len(segments), self.rmt_config.get("max_n_segments", 32)):
            output_ids, memory_state = self.generate_segment(memory_state=memory_state, **kwargs)
            generated_segments.append(output_ids)

            if self.all_done(generated_segments):
                break

        return generated_segments

    def manage_gradients(self, seg_num):
        k2, max_n_segments = self.rmt_config.get('k2', -1), self.rmt_config.get('max_n_segments')
        if seg_num == 0 \
            or k2 in {-1, None} \
                or seg_num + k2 > max_n_segments:
            return

        self.memory_cell.detach_memory_state()

    def process_outputs(self, cell_outputs, segments, **kwargs):
        out = CausalLMOutputWithCrossAttentions()
        proxy_out = {}
        for seg_num, segment in enumerate(segments):
            cell_out = cell_outputs[seg_num]

            full_logits = cell_out.logits

            labels = segment.get('labels')
            if labels is not None:
                shift_labels = labels[..., 1:].contiguous()
                shift_logits = full_logits[..., :-1, :].contiguous()
                flat_labels = shift_labels.view(-1)
                flat_logits = shift_logits.view(-1, shift_logits.size(-1))

                loss_fct = nn.CrossEntropyLoss()
                labels_mask = segment.get('labels_mask')
                if labels_mask is not None:
                    shift_mask = labels_mask[..., :-1].contiguous()

                    flat_labels = flat_labels[shift_mask.view(-1)]
                    flat_logits = flat_logits[shift_mask.view(-1)]

                    if labels_mask.sum() == 0:
                        loss_value = 0
                    else:
                        loss_value = loss_fct(flat_logits, flat_labels)

                proxy_out[f'loss_{seg_num}'] = loss_value
            else:
                proxy_out[f'loss_{seg_num}'] = 0

            segment_keys = ['loss']
            if kwargs.get('output_attentions'):
                segment_keys.append('attentions')
            if kwargs.get('output_hidden_states'):
                segment_keys.append('hidden_states')

            for key, value in cell_out.items():
                if any([sk in key for sk in segment_keys]):
                    proxy_out[f'{key}_{seg_num}'] = value

        num_segments = len(segments)
        out['loss'] = sum([proxy_out[f'loss_{seg_num}'] for seg_num in range(num_segments)]) / num_segments
        out['logits'] = torch.cat([cell_out.logits for cell_out in cell_outputs], dim=1)

        return out


In [6]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name)

In [10]:
memory_cell = RMCAMemoryCell(base_model, num_mem_tokens=4, layers_attr="transformer.h")
rmt = RMCAWrapperNoSegmentationGenerate(memory_cell, max_n_segments=32)

In [11]:
# base_model_1 = AutoModelForCausalLM.from_pretrained(model_name)
# test_in = tokenizer.encode("Hello world", return_tensors="pt")
# test_out = base_model_1(input_ids=test_in, output_hidden_states=True)
# for hs in test_out.hidden_states:
#     print(hs.mean().item(), hs.std().item(), hs.min().item(), hs.max().item())

In [12]:
# cpt_path = "/workspace-SR006.nfs2/bulatov/rmt/runs/multiplication_4x4/gpt2-rmca/1x1024_mem16_1024_LR1e-04-cot-v2/run_1/checkpoint-3500/pytorch_model.bin"
# cpt_path = "/workspace-SR006.nfs2/bulatov/rmt/runs/gsm8k-MS1/gpt2-rmca/1x1024_mem64_1024-cot-v2-curr-zero_mem/run1/checkpoint-800/pytorch_model.bin"
cpt_path = "/workspace-SR006.nfs2/bulatov/rmt/runs/gsm8k-MS1/gpt2-rmca/1x1024_mem4_1024-cot-v2-curr-zero_mem-wd5e-2/run1/checkpoint-1200/pytorch_model.bin"
cpt = torch.load(cpt_path, map_location='cpu')

rmt.load_state_dict(cpt)

RuntimeError: Error(s) in loading state_dict for RMCAWrapperNoSegmentationGenerate:
	Missing key(s) in state_dict: "memory_cell.model.transformer.h.0.layer.memory", "memory_cell.model.transformer.h.0.layer.layer.ln_1.weight", "memory_cell.model.transformer.h.0.layer.layer.ln_1.bias", "memory_cell.model.transformer.h.0.layer.layer.attn.c_attn.weight", "memory_cell.model.transformer.h.0.layer.layer.attn.c_attn.bias", "memory_cell.model.transformer.h.0.layer.layer.attn.c_proj.weight", "memory_cell.model.transformer.h.0.layer.layer.attn.c_proj.bias", "memory_cell.model.transformer.h.0.layer.layer.ln_2.weight", "memory_cell.model.transformer.h.0.layer.layer.ln_2.bias", "memory_cell.model.transformer.h.0.layer.layer.mlp.c_fc.weight", "memory_cell.model.transformer.h.0.layer.layer.mlp.c_fc.bias", "memory_cell.model.transformer.h.0.layer.layer.mlp.c_proj.weight", "memory_cell.model.transformer.h.0.layer.layer.mlp.c_proj.bias", "memory_cell.model.transformer.h.0.layer.mem_read_layer.ln.weight",

In [10]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

# Get the total number of parameters
count_parameters(base_model), count_parameters(rmt)

(181763328, 181763328)

In [11]:
# for n, p in rmt.named_parameters():
#     print(n, p.shape, p.requires_grad, p.grad.mean())

In [12]:
# out = rmt(**collated)

In [13]:
# out.loss.backward()

In [14]:
# layer.mem_read_layer.cross_attn

In [89]:
rmt.memory_cell.model.transformer.h[0].mem_read_layer

RMCrossAttention(
  (ln): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (cross_attn): GPT2Attention(
    (c_attn): Conv1D(nf=1536, nx=768)
    (q_attn): Conv1D(nf=768, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
)

In [90]:
for layer in rmt.memory_cell.model.transformer.h:
    layer.mem_read_layer.cross_attn.c_proj.weight.data.zero_()
    layer.mem_write_layer.cross_attn.c_proj.weight.data.zero_()
    # print(layer.mem_read_layer.cross_attn.c_proj.weight)

In [14]:
self = rmt
segments = collated['segments']
output_attentions = True
output_hidden_states = True


debug_states = {}
cell_outputs = []
for seg_num, segment in enumerate(segments):
    cell_out = self.memory_cell(input_ids=segment['input_ids'],
                                attention_mask=segment['attention_mask'],
                                output_attentions=output_attentions,
                                output_hidden_states=True)
    cell_outputs.append(cell_out)
    self.manage_gradients(seg_num)
    # break
    debug_states[seg_num] = [layer.debug_state for layer in rmt.memory_cell.model.transformer.h]
    debug_states[f"{seg_num}-read"] = [layer.mem_read_layer.debug_state for layer in rmt.memory_cell.model.transformer.h]
    debug_states[f"{seg_num}-write"] = [layer.mem_write_layer.debug_state for layer in rmt.memory_cell.model.transformer.h]
    

# out = self.process_outputs(cell_outputs, segments,
#                             output_attentions=output_attentions,
#                             output_hidden_states=output_hidden_states)

`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [15]:
len(collated['segments'])

3

In [16]:
debug_states.keys(), debug_states[0][0].keys(), debug_states['0-read'][0].keys()

(dict_keys([0, '0-read', '0-write', 1, '1-read', '1-write', 2, '2-read', '2-write']),
 dict_keys(['inp_hidden_states', 'mem_read_out', 'mem_write_out', 'memory_state', 'output', 'cross_attentions_read', 'cross_attentions_write']),
 dict_keys(['encoder_hidden_states', 'inp_hidden_states', 'output', 'attn_output', 'cross_attn_outputs']))

In [17]:
# layer_state.keys()

In [18]:
# len(layer_state[key])

In [19]:
seg_num = "0-read"
# seg_num = "0-write"

keys = ['inp_hidden_states', 'encoder_hidden_states', 'attn_output']#, 'cross_attn_outputs'][-1:]
segment_states = debug_states[seg_num]
for key in keys:
    print()
    print(key)
    for layer_state in segment_states:
        mean_ = layer_state[key].mean()
        std_ = layer_state[key].std()
        min_ = layer_state[key].min()
        max_ = layer_state[key].max()
        print(mean_.item(), std_.item(), min_.item(), max_.item())


inp_hidden_states
-0.0005578971467912197 0.17760612070560455 -4.4091796875 3.728515625
-0.006232638843357563 2.0028131008148193 -71.403564453125 104.01599884033203
-0.006312801968306303 3.349181652069092 -73.46725463867188 593.6646728515625
0.03301436826586723 11.159987449645996 -73.06352233886719 2476.620849609375
0.044311583042144775 11.908797264099121 -72.47608947753906 2634.2236328125
0.058057330548763275 12.55832290649414 -71.00603485107422 2774.500732421875
0.07055488973855972 12.982148170471191 -66.20497131347656 2867.1318359375
0.0876457616686821 13.284734725952148 -60.688133239746094 2922.619140625
0.11958087235689163 13.608667373657227 -55.6173095703125 2956.922607421875
0.13541331887245178 13.890575408935547 -54.81237030029297 2977.7724609375
0.16596980392932892 14.439210891723633 -79.15882110595703 2988.052978515625
0.2772412598133087 15.742889404296875 -151.67889404296875 2989.009033203125

encoder_hidden_states
0.009510686621069908 0.9964831471443176 -4.25 4.125
0.004937

In [20]:
seg_num = 0

keys = ['inp_hidden_states', 'memory_state']#, 'output']
segment_states = debug_states[seg_num]
for key in keys:
    print()
    print(key)
    for layer_state in segment_states:
        mean_ = layer_state[key].mean()
        std_ = layer_state[key].std()
        min_ = layer_state[key].min()
        max_ = layer_state[key].max()
        print(mean_.item(), std_.item(), min_.item(), max_.item())


inp_hidden_states
-0.0004702328587882221 0.17899121344089508 -4.4458794593811035 3.7651281356811523
-0.006465740501880646 2.001833915710449 -71.40843200683594 104.05224609375
-0.00620992248877883 3.348708391189575 -73.4843978881836 593.6775512695312
0.033060163259506226 11.159908294677734 -73.0713882446289 2476.62158203125
0.04442776367068291 11.908841133117676 -72.48859405517578 2634.23876953125
0.058194100856781006 12.558292388916016 -71.01464080810547 2774.5048828125
0.07068280130624771 12.982096672058105 -66.1973876953125 2867.130126953125
0.08767484873533249 13.284655570983887 -60.688568115234375 2922.61328125
0.11953415721654892 13.608614921569824 -55.6114501953125 2956.91943359375
0.1354111284017563 13.89045524597168 -54.79837417602539 2977.76708984375
0.16602084040641785 14.438973426818848 -79.14502716064453 2988.04638671875
0.277363121509552 15.742731094360352 -151.67041015625 2989.01025390625

memory_state
0.011711889877915382 1.0000689029693604 -4.303404331207275 4.17692995

In [22]:
seg_num = 2

keys = ['inp_hidden_states', 'memory_state']#, 'output']
segment_states = debug_states[seg_num]
for key in keys:
    print()
    print(key)
    for layer_state in segment_states:
        mean_ = layer_state[key].mean()
        std_ = layer_state[key].std()
        min_ = layer_state[key].min()
        max_ = layer_state[key].max()
        print(mean_.item(), std_.item(), min_.item(), max_.item())


inp_hidden_states
-0.0020966017618775368 0.2840779423713684 -4.481912612915039 3.8016772270202637
0.05370934680104256 3.8139808177948 -80.22399139404297 118.39396667480469
0.5015147924423218 16.520660400390625 -89.22883605957031 665.1481323242188
3.3518521785736084 83.21925354003906 -90.10358428955078 2538.81689453125
3.580336093902588 87.45240783691406 -90.94190216064453 2770.834228515625
3.732922315597534 90.58382415771484 -89.65648651123047 2937.01416015625
3.813560724258423 92.4358901977539 -83.88822937011719 3046.693115234375
3.85661244392395 93.54927825927734 -77.65959167480469 3111.789306640625
3.8736605644226074 94.2428970336914 -71.942626953125 3151.786376953125
3.8853845596313477 94.51702117919922 -64.370849609375 3176.25537109375
3.901632308959961 94.51990509033203 -54.785762786865234 3187.723876953125
3.9437255859375 94.15647888183594 -41.6655158996582 3191.5703125

memory_state
0.015733545646071434 1.0116857290267944 -4.403924465179443 4.273698329925537
-0.051594011485576

In [ ]:
seg_num = 7

keys = ['inp_hidden_states', 'memory_state']#, 'output']
segment_states = debug_states[seg_num]
for key in keys:
    print()
    print(key)
    for layer_state in segment_states:
        mean_ = layer_state[key].mean()
        std_ = layer_state[key].std()
        min_ = layer_state[key].min()
        max_ = layer_state[key].max()
        print(mean_.item(), std_.item(), min_.item(), max_.item())

NameError: name 'debug_states' is not defined

In [28]:
[(layer_state['hidden_states'].mean(), layer_state['hidden_states'].std()) for layer_state in debug_states[0]]

[(tensor(-0.0063, grad_fn=<MeanBackward0>),
  tensor(0.6333, grad_fn=<StdBackward0>)),
 (tensor(-0.5642, grad_fn=<MeanBackward0>),
  tensor(29.4408, grad_fn=<StdBackward0>)),
 (tensor(-1.4124, grad_fn=<MeanBackward0>),
  tensor(50.7379, grad_fn=<StdBackward0>)),
 (tensor(-1.1679, grad_fn=<MeanBackward0>),
  tensor(72.6272, grad_fn=<StdBackward0>)),
 (tensor(-0.2154, grad_fn=<MeanBackward0>),
  tensor(98.1661, grad_fn=<StdBackward0>)),
 (tensor(-1.2143, grad_fn=<MeanBackward0>),
  tensor(120.1388, grad_fn=<StdBackward0>)),
 (tensor(-1.6876, grad_fn=<MeanBackward0>),
  tensor(134.8322, grad_fn=<StdBackward0>)),
 (tensor(-2.3790, grad_fn=<MeanBackward0>),
  tensor(147.0923, grad_fn=<StdBackward0>)),
 (tensor(-2.9443, grad_fn=<MeanBackward0>),
  tensor(164.2453, grad_fn=<StdBackward0>)),
 (tensor(-4.4038, grad_fn=<MeanBackward0>),
  tensor(179.6345, grad_fn=<StdBackward0>)),
 (tensor(-3.0699, grad_fn=<MeanBackward0>),
  tensor(196.0588, grad_fn=<StdBackward0>)),
 (tensor(-4.7097, grad_fn=<

In [32]:
cell_out.keys()

odict_keys(['logits', 'past_key_values', 'hidden_states', 'attentions'])

In [20]:
[x.shape for x in cell_out.hidden_states]

[torch.Size([10, 115, 768]),
 torch.Size([10, 115, 768]),
 torch.Size([10, 115, 768]),
 torch.Size([10, 115, 768]),
 torch.Size([10, 115, 768]),
 torch.Size([10, 115, 768]),
 torch.Size([10, 115, 768]),
 torch.Size([10, 115, 768]),
 torch.Size([10, 115, 768]),
 torch.Size([10, 115, 768]),
 torch.Size([10, 115, 768]),
 torch.Size([10, 115, 768]),
 torch.Size([10, 115, 768])]

In [59]:
layer_ind = 0
layer = rmt.memory_cell.model.transformer.h[0]
layer.debug_state.keys()

dict_keys(['hidden_states', 'memory_state', 'output', 'cross_attentions_read', 'cross_attentions_write'])

In [12]:
# layer.debug_state['memory_state']

In [13]:
# len(layer.debug_state['cross_attentions_read'][0][0])

In [14]:
rmt.memory_cell.model.transformer.h[0].mem_read_layer.cross_attn

GPT2Attention(
  (c_attn): Conv1D(nf=1536, nx=768)
  (q_attn): Conv1D(nf=768, nx=768)
  (c_proj): Conv1D(nf=768, nx=768)
  (attn_dropout): Dropout(p=0.1, inplace=False)
  (resid_dropout): Dropout(p=0.1, inplace=False)
)

## Create data

In [11]:
class Holder:
    def __init__(self):
        pass
args = Holder()
# args.use_cot = False
args.num_mem_tokens = None
args.task_name = 'gsm8k'
# args.task_name = 'multiplication'

In [12]:
id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
think = tokenizer.encode('????')
ans = tokenizer.encode('!!!!')
eos = [tokenizer.eos_token_id]
if 'gsm8k' in args.task_name:
    delim = ">> <<"
elif 'multiplication' in args.task_name:
    delim = ' + '
else:
    raise NotImplementedError(f"Unknown task name {args.task_name}")

id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
bos = tokenizer.encode('////')
think = tokenizer.encode('????')
ans = tokenizer.encode('!!!!')
eos = [tokenizer.eos_token_id]
if 'gsm8k' in args.task_name:
    delim = ">> <<"
elif 'multiplication' in args.task_name:
    delim = ' + '
else:
    raise NotImplementedError(f"Unknown task name {args.task_name}")

def collate_fn(batch):
    # first, we segment each sample into task, cot steps and labels
    segments_batch = []
    for sample in batch:
        task, lab, cot = sample['task'], sample['labels'], sample['cot']
        task_tokens = tokenizer.encode(task, add_special_tokens=False)
        labels_tokens = tokenizer.encode(lab, add_special_tokens=False)
        cot_segments = split_cot(cot, by=delim)
        cot_segment_tokens = tokenizer.batch_encode_plus(cot_segments, add_special_tokens=False)['input_ids']

        segments = []
        segments.append(make_segment(bos + task_tokens + think, loss=False))
        for segment in cot_segment_tokens[:-1]:
            segments.append(make_segment(bos + segment + think, loss=True))
        segments.append(make_segment(bos + cot_segment_tokens[-1] + ans, loss=True))

        segments.append(make_segment(bos + labels_tokens + eos, loss=True))
        segments_batch.append(segments)

    # if some samples have less segments than others, we pad them with empty segments
    num_segments = max(len(segments) for segments in segments_batch)
    for segments in segments_batch:
        if len(segments) < num_segments:
            segments.extend([make_segment(eos, loss=False)] * (num_segments - len(segments)))

    # prepare segments for the whole batch
    batch_segments = []
    for i in range(num_segments):
        input_ids = [s[i]['input_ids'] for s in segments_batch]
        attention_mask = [s[i]['attention_mask'] for s in segments_batch]
        labels = [s[i]['labels'] for s in segments_batch]
        labels_mask = [s[i]['labels_mask'] for s in segments_batch]

        input_ids = pad_sequence(input_ids, batch_first=True, padding_value=id_pad_value)
        attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
        labels = pad_sequence(labels, batch_first=True, padding_value=-100)
        labels_mask = pad_sequence(labels_mask, batch_first=True, padding_value=False)

        batch_segment = {'input_ids': input_ids,
                            'attention_mask': attention_mask,
                            'labels_mask': labels_mask,
                            'labels': labels
                            }
        batch_segments.append(batch_segment)
    full_labels = torch.cat([s['labels'] for s in batch_segments], dim=1)
    return {"segments": batch_segments, 'labels': full_labels}


# dataset = 'booydar/gsm8k'
dataset = '/workspace-SR006.nfs2/bulatov/rmt/data/gsm8k'
# dataset = 'booydar/multiplication_4x4'
# dataset = f"booydar/{args.task_name}"
train_dataset = datasets.load_dataset(dataset, split='train')
valid_dataset = datasets.load_dataset(dataset, split='valid')

args.max_cot_steps = 1
if args.max_cot_steps is not None:
    train_dataset = train_dataset.filter(lambda x: x['cot_len'] <= args.max_cot_steps)
    valid_dataset = valid_dataset.filter(lambda x: x['cot_len'] <= args.max_cot_steps)
    # test_dataset = test_dataset.filter(lambda x: x['cot_len'] <= args.max_cot_steps)

In [13]:
batch = [valid_dataset[i] for i in range(10)]
collated = collate_fn(batch)

In [14]:
out = rmt(**collated, output_hidden_states=True, output_attentions=True)

`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


### debug

In [8]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained('gpt2')
base_model = AutoModelForCausalLM.from_config(config)

In [19]:
from transformers import GPT2LMHeadModel
from transformers.models.gpt2.modeling_gpt2 import GPT2Block, GPT2Attention

In [20]:
# block

In [10]:
block_config = config
block_config.add_cross_attention=True
block = GPT2Block(block_config)

In [17]:
config.hidden_size

768

In [ ]:
# my block
import torch.nn as nn



In [36]:
hidden_states = torch.rand(4, 32, config.n_embd)
ca_hidden_states = torch.rand(4, 10, config.n_embd)

In [37]:
out = block(hidden_states, encoder_hidden_states=ca_hidden_states)

In [39]:
len(out)

1

In [41]:
out[0].shape

torch.Size([4, 32, 768])

In [21]:
block.mlp.c_proj.weight

Parameter containing:
tensor([[-0.0185, -0.0107, -0.0151,  ..., -0.0018, -0.0174, -0.0430],
        [-0.0091, -0.0032,  0.0377,  ..., -0.0201,  0.0110,  0.0046],
        [ 0.0122, -0.0052, -0.0007,  ..., -0.0021,  0.0069, -0.0088],
        ...,
        [ 0.0200,  0.0150, -0.0071,  ...,  0.0020,  0.0151, -0.0020],
        [-0.0079, -0.0102, -0.0498,  ...,  0.0047, -0.0243,  0.0262],
        [-0.0123,  0.0150,  0.0093,  ..., -0.0046,  0.0151, -0.0227]],
       requires_grad=True)

In [9]:
base_model.transformer.wte.weight

Parameter containing:
tensor([[-0.0082,  0.0061,  0.0125,  ..., -0.0002, -0.0164, -0.0277],
        [-0.0287, -0.0139, -0.0143,  ...,  0.0064, -0.0097, -0.0254],
        [ 0.0210,  0.0078, -0.0080,  ..., -0.0346,  0.0157,  0.0078],
        ...,
        [ 0.0356, -0.0018,  0.0124,  ...,  0.0365, -0.0038, -0.0041],
        [-0.0207,  0.0009, -0.0061,  ..., -0.0011,  0.0134,  0.0031],
        [ 0.0092,  0.0036, -0.0126,  ...,  0.0296,  0.0002,  0.0204]],
       requires_grad=True)

In [ ]:
base_model

In [ ]:
base_model.transformer.wte.weight

Parameter containing:
tensor([[ 0.0182, -0.0330,  0.0011,  ..., -0.0458,  0.0089,  0.0296],
        [-0.0164,  0.0429,  0.0111,  ..., -0.0027,  0.0223, -0.0236],
        [-0.0230,  0.0143,  0.0186,  ..., -0.0436,  0.0020,  0.0011],
        ...,
        [ 0.0060, -0.0180,  0.0040,  ...,  0.0082,  0.0026, -0.0469],
        [-0.0089, -0.0230,  0.0076,  ...,  0.0394,  0.0006,  0.0032],
        [-0.0021,  0.0327, -0.0022,  ..., -0.0012, -0.0276,  0.0112]],
       requires_grad=True)

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [58]:
layer = model.transformer.h[0]

In [60]:
layer.attn

GPT2Attention(
  (c_attn): Conv1D(nf=2304, nx=768)
  (c_proj): Conv1D(nf=768, nx=768)
  (attn_dropout): Dropout(p=0.1, inplace=False)
  (resid_dropout): Dropout(p=0.1, inplace=False)
)

In [44]:
import copy
import torch.nn as nn

def clone_layer_fresh(layer: nn.Module, reset_weights=False) -> nn.Module:
    new_layer = copy.deepcopy(layer)

    def recursive_reset(module: nn.Module):
        for child in module.children():
            print(child)
            recursive_reset(child)
        # if hasattr(module, 'reset_parameters'):
            module.reset_parameters()
        
        return module

    if reset_weights:
        new_layer = recursive_reset(new_layer)
    return new_layer


In [ ]:
model.reset_paraa

In [46]:
layer.reset_parameters()

AttributeError: 'GPT2Block' object has no attribute 'reset_parameters'

In [45]:
cloned = clone_layer_fresh(layer, reset_weights=True)

LayerNorm((768,), eps=1e-05, elementwise_affine=True)


AttributeError: 'GPT2Block' object has no attribute 'reset_parameters'

In [40]:
layer.attn.c_attn.weight

Parameter containing:
tensor([[-0.4738, -0.2614, -0.0978,  ...,  0.0513, -0.0584,  0.0250],
        [ 0.0874,  0.1473,  0.2387,  ..., -0.0525, -0.0113, -0.0156],
        [ 0.0039,  0.0695,  0.3668,  ...,  0.1143,  0.0363, -0.0318],
        ...,
        [-0.2592, -0.0164,  0.1991,  ...,  0.0095, -0.0516,  0.0319],
        [ 0.1517,  0.2170,  0.1043,  ...,  0.0293, -0.0429, -0.0475],
        [-0.4100, -0.1924, -0.2400,  ..., -0.0046,  0.0070,  0.0198]],
       requires_grad=True)

In [41]:
cloned.attn.c_attn.weight

Parameter containing:
tensor([[-0.4738, -0.2614, -0.0978,  ...,  0.0513, -0.0584,  0.0250],
        [ 0.0874,  0.1473,  0.2387,  ..., -0.0525, -0.0113, -0.0156],
        [ 0.0039,  0.0695,  0.3668,  ...,  0.1143,  0.0363, -0.0318],
        ...,
        [-0.2592, -0.0164,  0.1991,  ...,  0.0095, -0.0516,  0.0319],
        [ 0.1517,  0.2170,  0.1043,  ...,  0.0293, -0.0429, -0.0475],
        [-0.4100, -0.1924, -0.2400,  ..., -0.0046,  0.0070,  0.0198]],
       requires_grad=True)

In [22]:
model.config == layer.attn.config

True

In [20]:
from transformers import AutoModel

In [21]:
AutoModel.from_config(layer.attn.config)

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [5]:
class Holder:
    def __init__(self):
        pass
args = Holder()
# args.use_cot = False
args.num_mem_tokens = None
# args.task_name = 'gsm8k'
args.task_name = 'multiplication'

In [11]:
id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
think = tokenizer.encode('????')
ans = tokenizer.encode('!!!!')
eos = [tokenizer.eos_token_id]
if 'gsm8k' in args.task_name:
    delim = ">> <<"
elif 'multiplication' in args.task_name:
    delim = ' + '
else:
    raise NotImplementedError(f"Unknown task name {args.task_name}")

id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
think = tokenizer.encode('????')
bos = tokenizer.encode('////')
ans = tokenizer.encode('!!!!')
eos = [tokenizer.eos_token_id]
if 'gsm8k' in args.task_name:
    delim = ">> <<"
elif 'multiplication' in args.task_name:
    delim = ' + '
else:
    raise NotImplementedError(f"Unknown task name {args.task_name}")

def collate_fn(batch):
    # first, we segment each sample into task, cot steps and labels
    segments_batch = []
    for sample in batch:
        task, lab, cot = sample['task'], sample['labels'], sample['cot']
        task_tokens = tokenizer.encode(task, add_special_tokens=False)
        labels_tokens = tokenizer.encode(lab, add_special_tokens=False)
        cot_segments = split_cot(cot, by=delim)
        cot_segment_tokens = tokenizer.batch_encode_plus(cot_segments, add_special_tokens=False)['input_ids']

        segments = []
        segments.append(make_segment(bos + task_tokens + think, loss=False))
        for segment in cot_segment_tokens[:-1]:
            segments.append(make_segment(bos + segment + think, loss=True))
        segments.append(make_segment(bos + cot_segment_tokens[-1] + ans, loss=True))

        segments.append(make_segment(bos + labels_tokens + eos, loss=True))
        segments_batch.append(segments)

    # if some samples have less segments than others, we pad them with empty segments
    num_segments = max(len(segments) for segments in segments_batch)
    for segments in segments_batch:
        if len(segments) < num_segments:
            segments.extend([make_segment(eos, loss=False)] * (num_segments - len(segments)))

    # prepare segments for the whole batch
    batch_segments = []
    for i in range(num_segments):
        input_ids = [s[i]['input_ids'] for s in segments_batch]
        attention_mask = [s[i]['attention_mask'] for s in segments_batch]
        labels = [s[i]['labels'] for s in segments_batch]
        labels_mask = [s[i]['labels_mask'] for s in segments_batch]

        input_ids = pad_sequence(input_ids, batch_first=True, padding_value=id_pad_value)
        attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
        labels = pad_sequence(labels, batch_first=True, padding_value=-100)
        labels_mask = pad_sequence(labels_mask, batch_first=True, padding_value=False)

        batch_segment = {'input_ids': input_ids,
                            'attention_mask': attention_mask,
                            'labels_mask': labels_mask,
                            'labels': labels
                            }
        batch_segments.append(batch_segment)
    full_labels = torch.cat([s['labels'] for s in batch_segments], dim=1)
    return {"segments": batch_segments, 'labels': full_labels}


In [12]:
# dataset = 'booydar/gsm8k'
dataset = 'booydar/multiplication_4x4'
# dataset = f"booydar/{args.task_name}"
train_dataset = datasets.load_dataset(dataset, split='train')
valid_dataset = datasets.load_dataset(dataset, split='valid')

Using the latest cached version of the dataset since booydar/multiplication_4x4 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/jovyan/.cache/huggingface/datasets/booydar___multiplication_4x4/default/0.0.0/e224243bb9970a0937976173287d24816d683ff9 (last modified on Fri Mar 14 16:54:33 2025).
Using the latest cached version of the dataset since booydar/multiplication_4x4 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/jovyan/.cache/huggingface/datasets/booydar___multiplication_4x4/default/0.0.0/e224243bb9970a0937976173287d24816d683ff9 (last modified on Fri Mar 14 16:54:33 2025).


In [13]:
valid_dataset[0]

{'task': '5 6 3 2 * 7 4 3 4',
 'labels': '5 5 6 0 8 2 0 1',
 'cot': '5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0'}

In [5]:
model_name = 'gpt2'
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
from transformers.modeling_utils

In [ ]:
model.generate

AttributeError: 'GPT2LMHeadModel' object has no attribute 'ge'

In [ ]:

memory_cell = MemoryCell(
    model,
    num_mem_tokens=16
)
rmt = RecurrentWrapperNoSegmentation(memory_cell, max_n_segments=32)

In [ ]:
# checkpoint_path = "/workspace-SR006.nfs2/bulatov/rmt/runs/gsm8k/gpt2/SEGM_1x1024_1024_LR3e-04-cot/checkpoint-17500/pytorch_model.bin"
# checkpoint_path = "/workspace-SR006.nfs2/bulatov/rmt/runs/multiplication_4x4/gpt2/1x1024_mem4_1024_LR6e-04/checkpoint-124500/pytorch_model.bin"
# checkpoint_path = "/workspace-SR006.nfs2/bulatov/rmt/runs/multiplication_4x4/gpt2/1x1024_mem16_1024_LR1e-03-v2/checkpoint-78000/pytorch_model.bin"
# checkpoint_path = "/workspace-SR006.nfs2/bulatov/rmt/runs/multiplication_4x4/gpt2/1x1024_mem16_1024_LR1e-03-cot-v2/checkpoint-60000/pytorch_model.bin"
# checkpoint_path = "/workspace-SR006.nfs2/bulatov/rmt/runs/multiplication_4x4/gpt2/1x1024_mem16_1024_LR1e-03-cot-v2/checkpoint-90000/pytorch_model.bin"

# checkpoint_path = "/workspace-SR006.nfs2/bulatov/rmt/runs/gsm8k/gpt2/1x1024_mem4_1024_LR3e-03-cot-v2/checkpoint-47500/pytorch_model.bin"
device = 'cpu'

rmt.load_state_dict(torch.load(checkpoint_path), strict=False)
# rmt.generation_config.pad_token_id = tokenizer.pad_token_id
rmt.to(device)
print(':)')

:)


In [10]:
args.use_cot = True

In [11]:

collated = collate_fn([sample for sample in valid_dataset.select(range(128))])
# collated = collate_fn([sample for sample in train_dataset.select(range(16))])

In [12]:
segments = collated['segments']

In [13]:
# tokenizer.batch_decode(segments[1]['input_ids'])

In [14]:
for i in range(len(segments)):
    print(segments[i]['input_ids'].shape, segments[i]['labels'].shape)

torch.Size([128, 11]) torch.Size([128, 11])
torch.Size([128, 7]) torch.Size([128, 7])
torch.Size([128, 16]) torch.Size([128, 16])
torch.Size([128, 18]) torch.Size([128, 18])
torch.Size([128, 10]) torch.Size([128, 10])
torch.Size([128, 10]) torch.Size([128, 10])


In [15]:
# out = rmt(**collated)
self = rmt
segments = collated['segments']

memory_state = None

cell_outputs = []
for seg_num, segment in enumerate(segments):
    cell_out, memory_state = self.memory_cell(input_ids=segment['input_ids'],
                                                attention_mask=segment['attention_mask'],
                                                # labels=segment['input_ids'],

                                                memory_state=memory_state, 
                                                output_hidden_states=True)
    cell_outputs.append(cell_out)
    self.manage_gradients(memory_state, seg_num)

# out = self.process_outputs(cell_outputs, segments,
#                             output_attentions=output_attentions,
#                             output_hidden_states=output_hidden_states)

In [18]:
out = dict()
from torch.nn import CrossEntropyLoss
self = rmt
kwargs = {}

proxy_out = {}
for seg_num, segment in enumerate(segments):
    cell_out = cell_outputs[seg_num]

    full_logits = cell_out.logits

    labels = segment.get('labels')
    if labels is not None:
        shift_labels = labels[..., 1:].contiguous()
        shift_logits = full_logits[..., :-1, :].contiguous()
        flat_labels = shift_labels.view(-1)
        flat_logits = shift_logits.view(-1, shift_logits.size(-1))

        loss_fct = CrossEntropyLoss()
        labels_mask = segment.get('labels_mask')
        if labels_mask is not None:
            shift_mask = labels_mask[..., :-1].contiguous()

            flat_labels = flat_labels[shift_mask.view(-1)]
            flat_logits = flat_logits[shift_mask.view(-1)]

            if labels_mask.sum() == 0:
                loss_value = 0
            else:
                loss_value = loss_fct(flat_logits, flat_labels)

        proxy_out[f'loss_{seg_num}'] = loss_value
    else:
        proxy_out[f'loss_{seg_num}'] = 0

    segment_keys = ['loss']
    if kwargs.get('output_attentions'):
        segment_keys.append('attentions')
    if kwargs.get('output_hidden_states'):
        segment_keys.append('hidden_states')

    for key, value in cell_out.items():
        if any([sk in key for sk in segment_keys]):
            proxy_out[f'{key}_{seg_num}'] = value

num_segments = len(segments)
out['loss'] = sum([proxy_out[f'loss_{seg_num}'] for seg_num in range(num_segments)]) / num_segments
out['logits'] = torch.cat([cell_out.logits for cell_out in cell_outputs], dim=1)

In [19]:
for k, v in proxy_out.items():
    print(k, v.item() if isinstance(v, torch.Tensor) else v)

loss_0 0
loss_1 1.2404494782458642e-06
loss_2 1.437594346498372e-05
loss_3 2.0184384993626736e-05
loss_4 2.69055385615502e-06
loss_5 0.2612733542919159


In [68]:
valid_dataset[0]

{'task': '5 6 3 2 * 7 4 3 4',
 'labels': '5 5 6 0 8 2 0 1',
 'cot': '5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0'}

In [21]:

def compute_accuracy(eval_pred):
    preds = eval_pred.predictions.argmax(axis=-1)[:, :-1]
    labels = eval_pred.label_ids[:, 1:]

    labels_masks = labels > 0
    preds_full = [p[m] for p, m in zip(preds, labels_masks)]
    labels_full = [lab[m] for lab, m in zip(labels, labels_masks)]

    special_tokens = {ans[0], bos[0]}
    acc_cot, acc_ans = [], []
    for lab_tokens, pred_tokens in zip(labels_full, preds_full):
        ans_start_index = max(i for i, x in enumerate(lab_tokens) if x == ans[0])

        pred_cot_tokens = pred_tokens[:ans_start_index].tolist()
        lab_cot_tokens = lab_tokens[:ans_start_index].tolist()

        cot_correct = [p == l for p, l in zip(pred_cot_tokens, lab_cot_tokens) if l not in special_tokens]
        acc_cot.append(all(cot_correct))

        pred_ans_tokens = pred_tokens[ans_start_index:].tolist()
        lab_ans_tokens = lab_tokens[ans_start_index:].tolist()

        ans_correct = [p == l for p, l in zip(pred_ans_tokens, lab_ans_tokens) if l not in special_tokens]
        acc_ans.append(all(ans_correct))

    return {'accuracy_cot': np.mean(acc_cot), 'accuracy_ans': np.mean(acc_ans)}

In [22]:
special_tokens = {ans[0], bos[0]}


In [32]:
tokenizer.decode([p for p, l in zip(pred_ans_tokens, lab_ans_tokens) if l not in special_tokens])

'9 1 0 3 2 8 6 0<|endoftext|>'

In [33]:
tokenizer.decode([l for p, l in zip(pred_ans_tokens, lab_ans_tokens) if l not in special_tokens])

'9 1 0 3 0 8 6 0<|endoftext|>'

In [34]:
tokenizer.decode([l for p, l in zip(pred_ans_tokens, lab_ans_tokens)])

'!!!!////9 1 0 3 0 8 6 0<|endoftext|>'

In [35]:
tokenizer.decode([p for p, l in zip(pred_ans_tokens, lab_ans_tokens)])

'!!!! 69 1 0 3 2 8 6 0<|endoftext|>'

In [27]:
eval_pred = Holder()

eval_pred.predictions = out['logits']
eval_pred.label_ids = collated['labels']

In [28]:
acc = compute_accuracy(eval_pred)
print(acc)

{'accuracy_cot': 1.0, 'accuracy_ans': 0.0625}


In [58]:
preds = eval_pred.predictions.argmax(axis=-1)[:, :-1]
labels = eval_pred.label_ids[:, 1:]

labels_masks = labels > 0
preds_full = [p[m] for p, m in zip(preds, labels_masks)]
labels_full = [lab[m] for lab, m in zip(labels, labels_masks)]

acc_cot, acc_ans = [], []
correct_samples = []
for lab_tokens, pred_tokens in zip(labels_full, preds_full):
    ans_start_index = max(i for i, x in enumerate(lab_tokens) if x == ans[0])

    pred_cot_tokens = pred_tokens[:ans_start_index].tolist()
    lab_cot_tokens = lab_tokens[:ans_start_index].tolist()

    cot_correct = [p == l for p, l in zip(pred_cot_tokens, lab_cot_tokens) if l != bos[0]]
    acc_cot.append(all(cot_correct))

    pred_ans_tokens = pred_tokens[ans_start_index:].tolist()
    lab_ans_tokens = lab_tokens[ans_start_index:].tolist()

    # ans_correct = [p == l for p, l in zip(pred_ans_tokens, lab_ans_tokens)]
    ans_correct = [p == l for p, l in zip(pred_ans_tokens, lab_ans_tokens) if l not in special_tokens]
    acc_ans.append(all(ans_correct))
    if all(ans_correct):
        correct_samples.append(lab_tokens)
    # break


In [59]:
tokenizer.batch_decode(correct_samples)

['////1 9 7 9 1????////0 0 0 0 0 0 ( 1 9 7 9 1 0 )????////0 0 9 9 1 2 0 ( 1 9 6 9 3 2 0 )????////0 0 0 3 9 3 5 1!!!!////1 9 6 2 3 6 5 1<|endoftext|>',
 '////4 7 3 4 0????////0 2 2 1 3 1 ( 4 9 5 5 3 1 )????////0 0 8 4 7 8 0 ( 4 9 3 0 1 0 1 )????////0 0 0 8 4 7 8 0!!!!////4 9 3 8 5 7 9 0<|endoftext|>',
 '////0 0 2 6 2????////0 0 5 5 6 0 ( 0 0 7 1 9 0 )????////0 0 0 0 2 6 2 ( 0 0 7 1 1 7 2 )????////0 0 0 0 0 2 6 2!!!!////0 0 7 1 1 9 8 2<|endoftext|>',
 '////9 3 1 1 4????////0 2 4 1 9 0 ( 9 5 5 2 3 1 )????////0 0 3 1 7 3 1 ( 9 5 8 3 0 5 1 )????////0 0 0 5 5 8 2 2!!!!////9 5 8 8 5 3 4 2<|endoftext|>',
 '////5 4 9 7 4????////0 2 1 7 6 7 ( 5 6 0 5 1 8 )????////0 0 6 5 3 8 3 ( 5 6 6 0 5 6 4 )????////0 0 0 2 1 7 6 7!!!!////5 6 6 2 6 3 1 8<|endoftext|>',
 '////8 4 6 3 5????////0 2 1 4 3 1 ( 8 6 7 7 8 1 )????////0 0 8 1 1 0 2 ( 8 6 5 9 9 1 2 )????////0 0 0 4 5 3 0 6!!!!////8 6 5 3 5 5 2 6<|endoftext|>',
 '////6 1 2 9 0????////0 0 2 5 1 1 ( 6 1 4 4 2 1 )????////0 0 0 0 0 0 0 ( 6 1 4 4 2 1 0 )????/

In [60]:
tokenizer.batch_decode(labels_full[:5])

['////5 5 5 6 1????////0 0 6 4 9 0 ( 5 5 1 1 1 1 )????////0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 )????////0 0 0 0 6 4 9 0!!!!////5 5 6 0 8 2 0 1<|endoftext|>',
 '////6 7 1 1 3????////0 4 8 7 0 2 ( 6 1 0 9 3 2 )????////0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 )????////0 0 0 2 7 3 6 3!!!!////6 1 4 9 8 6 8 3<|endoftext|>',
 '////8 0 0 5 7????////0 4 8 3 4 8 ( 8 4 8 8 1 9 )????////0 0 6 7 3 9 0 ( 8 4 4 6 5 8 1 )????////0 0 0 2 3 6 5 6!!!!////8 4 4 8 8 4 7 6<|endoftext|>',
 '////9 0 9 2 1????////0 2 1 2 7 1 ( 9 2 0 5 8 1 )????////0 0 8 1 8 5 2 ( 9 2 8 6 6 7 2 )????////0 0 0 5 1 5 1 2!!!!////9 2 8 1 8 2 4 2<|endoftext|>',
 '////0 4 6 8 5????////0 0 5 6 6 3 ( 0 4 1 5 2 4 )????////0 0 0 8 9 3 4 ( 0 4 1 3 2 8 4 )????////0 0 0 0 5 6 6 3!!!!////0 4 1 3 7 4 1 4<|endoftext|>']

In [61]:
tokenizer.batch_decode(preds_full[:5])

[' ty5 5 5 6 1????????0 0 6 4 9 0 ( 5 5 1 1 1 1 )????????0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 )????????0 0 0 0 6 4 9 0!!!! 05 5 6 0 1 2 0 1<|endoftext|>',
 ' deposit6 7 1 1 3????????0 4 8 7 0 2 ( 6 1 0 9 3 2 )????????0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 )????????0 0 0 2 7 3 6 3!!!! 26 1 4 9 5 6 8 3<|endoftext|>',
 ' Bild8 0 0 5 7????????0 4 8 3 4 8 ( 8 4 8 8 1 9 )????????0 0 6 7 3 9 0 ( 8 4 4 6 5 8 1 )????????0 0 0 2 3 6 5 6!!!! 28 4 4 8 5 4 7 6<|endoftext|>',
 ' Hein9 0 9 2 1????????0 2 1 2 7 1 ( 9 2 0 5 8 1 )????????0 0 8 1 8 5 2 ( 9 2 8 6 6 7 2 )????????0 0 0 5 1 5 1 2!!!! 59 2 8 1 3 2 4 2<|endoftext|>',
 ' fragrance0 4 6 8 5????????0 0 5 6 6 3 ( 0 4 1 5 2 4 )????????0 0 0 8 9 3 4 ( 0 4 1 3 2 8 4 )????????0 0 0 0 5 6 6 3!!!! 00 4 1 3 9 4 1 4<|endoftext|>']

In [36]:
tokenizer.decode(labels_full[0])

'////5 5 5 6 1????////0 0 6 4 9 0 ( 5 5 1 1 1 1 )????////0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 )????////0 0 0 0 6 4 9 0!!!!////5 5 6 0 8 2 0 1<|endoftext|>'

In [37]:
tokenizer.decode(lab_tokens)

'////9 9 0 9 0????////0 2 3 1 2 1 ( 9 1 4 0 3 1 )????////0 0 6 6 0 6 0 ( 9 1 0 7 3 7 0 )????////0 0 0 6 6 0 6 0!!!!////9 1 0 3 0 8 6 0<|endoftext|>'

In [24]:
tokenizer.decode(ans[0])

'!!!!'

In [25]:
ans_start_index

50

In [38]:
tokenizer.batch_decode(preds_full[:5])

[' ty5 5 5 6 1????????0 0 6 4 9 0 ( 5 5 1 1 1 1 )????????0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 )????????0 0 0 0 6 4 9 0!!!! 05 5 6 0 1 2 0 1<|endoftext|>',
 ' deposit6 7 1 1 3????????0 4 8 7 0 2 ( 6 1 0 9 3 2 )????????0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 )????????0 0 0 2 7 3 6 3!!!! 26 1 4 9 5 6 8 3<|endoftext|>',
 ' Bild8 0 0 5 7????????0 4 8 3 4 8 ( 8 4 8 8 1 9 )????????0 0 6 7 3 9 0 ( 8 4 4 6 5 8 1 )????????0 0 0 2 3 6 5 6!!!! 28 4 4 8 5 4 7 6<|endoftext|>',
 ' Hein9 0 9 2 1????????0 2 1 2 7 1 ( 9 2 0 5 8 1 )????????0 0 8 1 8 5 2 ( 9 2 8 6 6 7 2 )????????0 0 0 5 1 5 1 2!!!! 59 2 8 1 3 2 4 2<|endoftext|>',
 ' fragrance0 4 6 8 5????????0 0 5 6 6 3 ( 0 4 1 5 2 4 )????????0 0 0 8 9 3 4 ( 0 4 1 3 2 8 4 )????????0 0 0 0 5 6 6 3!!!! 00 4 1 3 9 4 1 4<|endoftext|>']

In [39]:
tokenizer.batch_decode(labels_full[:5])

['////5 5 5 6 1????////0 0 6 4 9 0 ( 5 5 1 1 1 1 )????////0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 )????////0 0 0 0 6 4 9 0!!!!////5 5 6 0 8 2 0 1<|endoftext|>',
 '////6 7 1 1 3????////0 4 8 7 0 2 ( 6 1 0 9 3 2 )????////0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 )????////0 0 0 2 7 3 6 3!!!!////6 1 4 9 8 6 8 3<|endoftext|>',
 '////8 0 0 5 7????////0 4 8 3 4 8 ( 8 4 8 8 1 9 )????////0 0 6 7 3 9 0 ( 8 4 4 6 5 8 1 )????////0 0 0 2 3 6 5 6!!!!////8 4 4 8 8 4 7 6<|endoftext|>',
 '////9 0 9 2 1????////0 2 1 2 7 1 ( 9 2 0 5 8 1 )????////0 0 8 1 8 5 2 ( 9 2 8 6 6 7 2 )????////0 0 0 5 1 5 1 2!!!!////9 2 8 1 8 2 4 2<|endoftext|>',
 '////0 4 6 8 5????////0 0 5 6 6 3 ( 0 4 1 5 2 4 )????////0 0 0 8 9 3 4 ( 0 4 1 3 2 8 4 )????////0 0 0 0 5 6 6 3!!!!////0 4 1 3 7 4 1 4<|endoftext|>']

In [65]:
tokenizer.batch_decode([s[-9:] for s in preds_full[:5]])

['5 5 6 0 1 2 0 1<|endoftext|>',
 '6 1 4 9 5 6 8 3<|endoftext|>',
 '8 4 4 8 5 4 7 6<|endoftext|>',
 '9 2 8 1 3 2 4 2<|endoftext|>',
 '0 4 1 3 9 4 1 4<|endoftext|>']

In [66]:
tokenizer.batch_decode([s[-9:] for s in labels_full[:5]])

['5 5 6 0 8 2 0 1<|endoftext|>',
 '6 1 4 9 8 6 8 3<|endoftext|>',
 '8 4 4 8 8 4 7 6<|endoftext|>',
 '9 2 8 1 8 2 4 2<|endoftext|>',
 '0 4 1 3 7 4 1 4<|endoftext|>']

In [40]:
tokenizer.decode([p for p, l in zip(pred_cot_tokens, lab_cot_tokens) if l != bos[0]])

'9 9 0 9 0????0 2 3 1 2 1 ( 9 1 4 0 3 1 )????0 0 6 6 0 6 0 ( 9 1 0 7 3 7 0 )????0 0 0 6 6 0 6 0'

In [41]:
tokenizer.decode([l for p, l in zip(pred_cot_tokens, lab_cot_tokens) if l != bos[0]])

'9 9 0 9 0????0 2 3 1 2 1 ( 9 1 4 0 3 1 )????0 0 6 6 0 6 0 ( 9 1 0 7 3 7 0 )????0 0 0 6 6 0 6 0'

In [167]:
cot_correct

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [16]:
# np.mean([value.item() if isinstance(value, torch.Tensor) else value  for value in proxy_out.values()])

In [17]:
cell_outputs[0].keys()

odict_keys(['logits', 'hidden_states'])

In [18]:
si = 1

In [19]:
# inputs
for seg in segments[:]:
    print(tokenizer.decode(seg['input_ids'][si]))

////6 9 1 5 * 6 4 4 7????
////6 7 1 1 3????
////0 4 8 7 0 2 ( 6 1 0 9 3 2 )????
////0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 )????
////0 0 0 2 7 3 6 3!!!!
////6 1 4 9 8 6 8 3<|endoftext|>


In [20]:
# predictions
[tokenizer.decode(o.logits.argmax(dim=-1)[si]) for o in cell_outputs]

['robees buttons 2 4 5- 2 dunk wellnesspard',
 '6 7 1 1 3????????',
 '0 4 8 7 0 2 ( 6 1 0 9 3 2 )???? 9',
 '0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 )???? 7',
 '0 0 0 2 7 3 6 3!!!!!!!!',
 '6 1 4 9 1 6 8 3<|endoftext|><|endoftext|>']

In [ ]:
def compute_accuracy(eval_pred):
    preds = eval_pred.predictions.argmax(axis=-1)[:, :-1]
    labels = eval_pred.label_ids[:, 1:]

    labels_masks = labels > 0
    preds_full = [p[m] for p, m in zip(preds, labels_masks)]
    labels_full = [lab[m] for lab, m in zip(labels, labels_masks)]

    special_tokens = {ans[0], bos[0]}
    acc_cot, acc_ans = [], []
    for lab_tokens, pred_tokens in zip(labels_full, preds_full):
        ans_start_index = max(i for i, x in enumerate(lab_tokens) if x == ans[0])

        pred_cot_tokens = pred_tokens[:ans_start_index].tolist()
        lab_cot_tokens = lab_tokens[:ans_start_index].tolist()

        cot_correct = [p == l for p, l in zip(pred_cot_tokens, lab_cot_tokens) if l not in special_tokens]
        acc_cot.append(all(cot_correct))

        pred_ans_tokens = pred_tokens[ans_start_index:].tolist()
        lab_ans_tokens = lab_tokens[ans_start_index:].tolist()

        ans_correct = [p == l for p, l in zip(pred_ans_tokens, lab_ans_tokens)]
        acc_ans.append(all(ans_correct))

    return {'accuracy_cot': np.mean(acc_cot), 'accuracy_ans': np.mean(acc_ans)}

tensor([[ -100,  -100,  -100,  ...,   657,   352, 50256],
        [ -100,  -100,  -100,  ...,   807,   513, 50256],
        [ -100,  -100,  -100,  ...,   767,   718, 50256],
        ...,
        [ -100,  -100,  -100,  ...,   807,   657, 50256],
        [ -100,  -100,  -100,  ...,   642,   352, 50256],
        [ -100,  -100,  -100,  ...,   604,   807, 50256]])

In [41]:
preds = out['logits'].argmax(axis=-1)[:, :-1]
labels = collated['labels'][:, 1:]
labels_masks = labels > 0

In [42]:
preds_full = [p[m] for p, m in zip(preds, labels_masks)]
labels_full = [lab[m] for lab, m in zip(labels, labels_masks)]

print(len(preds_full), len(labels_full))

preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

16 16


In [54]:
think

[9805]

In [ ]:
preds_full[0]

AttributeError: 'Tensor' object has no attribute 'find'

In [ ]:
def get_ans(tensor):
    tokens = tensor.tolist()
    
    try:
        start_index = max(i for i, x in enumerate(tokens) if x == ans[0])
    except ValueError:
        return []
    
    tokens = tokens[start_index + 1:]
    
    if tokens and tokens[0] == bos[0]:
        tokens = tokens[1:]
    
    return torch.tensor(tokens)

def get_cot(tensor)

In [109]:
tokens = labels_full[0].tolist()

ans_start_index = max(i for i, x in enumerate(tokens) if x == ans[0])



In [145]:
lab_tokens = labels_full[0]
pred_tokens = preds_full[0]

special_tokens = {ans[0], bos[0]}
pred_cot_tokens = pred_tokens[:ans_start_index].tolist()
lab_cot_tokens = lab_tokens[:ans_start_index].tolist()

cot_correct = [p == l for p, l in zip(pred_cot_tokens, lab_cot_tokens) if l not in special_tokens]

pred_ans_tokens = pred_tokens[ans_start_index:].tolist()
lab_ans_tokens = lab_tokens[ans_start_index:].tolist()

ans_correct = [p == l for p, l in zip(pred_ans_tokens, lab_ans_tokens)]

In [147]:
np.mean(cot_correct)

1.0

In [148]:
np.mean(ans_correct)

0.8181818181818182

In [142]:
tokenizer.decode(pred_tokens)

' 45 5 5 6 1????????0 0 6 4 9 0 ( 5 5 1 1 1 1 )????????0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 )???? 00 0 0 0 6 4 9 0!!!!!!!!5 5 6 0 7 2 0 1<|endoftext|>'

In [ ]:
# preds
tokenizer.decode([p for p, l in zip(pred_cot_tokens, lab_cot_tokens) if l not in special_tokens])

'5 5 5 6 1????0 0 6 4 9 0 ( 5 5 1 1 1 1 )????0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 )????0 0 0 0 6 4 9 0'

In [ ]:
# labels
tokenizer.decode([l for p, l in zip(pred_cot_tokens, lab_cot_tokens) if l not in special_tokens])

'5 5 5 6 1????0 0 6 4 9 0 ( 5 5 1 1 1 1 )????0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 )????0 0 0 0 6 4 9 0'

In [ ]:
[l for l in lab_cot_tokens if l not in special_tokens]

[9805, 9805, 9805]

In [131]:
think[0]

9805

In [132]:
pred_cot_tokens.shape

AttributeError: 'list' object has no attribute 'shape'

In [133]:
len(cot_correct)

47

In [107]:
# tokens

In [98]:
len(get_ans(preds_full[0])), len(get_ans(labels_full[0])), 

(9, 9)

In [99]:
tokenizer.decode(get_ans(preds_full[0]))

'5 5 6 0 7 2 0 1<|endoftext|>'

In [100]:
tokenizer.decode(get_ans(labels_full[0]))

'5 5 6 0 8 2 0 1<|endoftext|>'

In [69]:
def get_ans(text):
    splits = text.split(ans_text)
    splits = list(filter(len, splits))
    ans = splits[-1].strip()
    if ans.startswith(ans_text):
        ans = ans[len(ans_text):]
    if ans.startswith(bos_text):
        ans = ans[len(bos_text):]
    return ans

In [70]:
get_ans(preds_full_text[0]), get_ans(labels_full_text[0])

('5 5 6 0 7 2 0 1<|endoftext|>', '5 5 6 0 8 2 0 1<|endoftext|>')

In [71]:
preds_full_text[0].split(ans_text)

[' 45 5 5 6 1????????0 0 6 4 9 0 ( 5 5 1 1 1 1 )????????0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 )???? 00 0 0 0 6 4 9 0',
 '',
 '5 5 6 0 7 2 0 1<|endoftext|>']

In [ ]:
labels_full_text[0].split(ans_text)

['////5 5 5 6 1????////0 0 6 4 9 0 ( 5 5 1 1 1 1 )????////0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 )????////0 0 0 0 6 4 9 0',
 '////5 5 6 0 8 2 0 1<|endoftext|>']

In [65]:
preds_full_text

[' 45 5 5 6 1????????0 0 6 4 9 0 ( 5 5 1 1 1 1 )????????0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 )???? 00 0 0 0 6 4 9 0!!!!!!!!5 5 6 0 7 2 0 1<|endoftext|>',
 'pard6 7 1 1 3????????0 4 8 7 0 2 ( 6 1 0 9 3 2 )???? 90 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 )???? 70 0 0 2 7 3 6 3!!!!!!!!6 1 4 9 1 6 8 3<|endoftext|>',
 ' Doe8 0 0 5 7????????0 4 8 3 4 8 ( 8 4 8 8 1 9 )????????0 0 6 7 3 9 0 ( 8 4 4 6 5 8 1 )???? 60 0 0 2 3 6 5 6!!!!!!!!8 4 4 8 1 4 7 6<|endoftext|>',
 ' Gentleman9 0 9 2 1????????0 2 1 2 7 1 ( 9 2 0 5 8 1 )???? 50 0 8 1 8 5 2 ( 9 2 8 6 6 7 2 )???? 60 0 0 5 1 5 1 2!!!!!!!!9 2 8 1 0 2 4 2<|endoftext|>',
 ' drum0 4 6 8 5????????0 0 5 6 6 3 ( 0 4 1 5 2 4 )???? 50 0 0 8 9 3 4 ( 0 4 1 3 2 8 4 )???? 30 0 0 0 5 6 6 3!!!!!!!!0 4 1 3 9 4 1 4<|endoftext|>',
 ' 42 5 2 4 2????????0 9 8 1 8 1 ( 2 4 1 6 0 2 )???? 60 0 4 0 5 8 4 ( 2 4 5 6 5 0 5 )???? 60 0 0 1 4 4 2 4!!!!!!!!2 4 5 7 1 4 7 4<|endoftext|>',
 'hovah8 4 7 9 0????????0 6 6 8 3 4 ( 8 0 4 8 4 4 )???? 80 0 4 7 8 4 0 ( 8 0 8 5 3 9 0 )???? 50 0 0 4 4 2 9 2!!

In [48]:
labels_full_text

['////5 5 5 6 1????////0 0 6 4 9 0 ( 5 5 1 1 1 1 )????////0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 )????////0 0 0 0 6 4 9 0!!!!////5 5 6 0 8 2 0 1<|endoftext|>',
 '////6 7 1 1 3????////0 4 8 7 0 2 ( 6 1 0 9 3 2 )????////0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 )????////0 0 0 2 7 3 6 3!!!!////6 1 4 9 8 6 8 3<|endoftext|>',
 '////8 0 0 5 7????////0 4 8 3 4 8 ( 8 4 8 8 1 9 )????////0 0 6 7 3 9 0 ( 8 4 4 6 5 8 1 )????////0 0 0 2 3 6 5 6!!!!////8 4 4 8 8 4 7 6<|endoftext|>',
 '////9 0 9 2 1????////0 2 1 2 7 1 ( 9 2 0 5 8 1 )????////0 0 8 1 8 5 2 ( 9 2 8 6 6 7 2 )????////0 0 0 5 1 5 1 2!!!!////9 2 8 1 8 2 4 2<|endoftext|>',
 '////0 4 6 8 5????////0 0 5 6 6 3 ( 0 4 1 5 2 4 )????////0 0 0 8 9 3 4 ( 0 4 1 3 2 8 4 )????////0 0 0 0 5 6 6 3!!!!////0 4 1 3 7 4 1 4<|endoftext|>',
 '////2 5 2 4 2????////0 9 8 1 8 1 ( 2 4 1 6 0 2 )????////0 0 4 0 5 8 4 ( 2 4 5 6 5 0 5 )????////0 0 0 1 4 4 2 4!!!!////2 4 5 7 9 4 7 4<|endoftext|>',
 '////8 4 7 9 0????////0 6 6 8 3 4 ( 8 0 4 8 4 4 )????////0 0 4 7 8 4 0 ( 8 0 8 5 3 9 0 )????/

In [ ]:

preds_cot = [extract_cot(p) for p in preds_full_text]
preds_ans = [extract_answer(p) for p in preds_full_text]

labels_cot = [extract_cot(lab) for lab in labels_full_text]
labels_ans = [extract_answer(lab) for lab in labels_full_text]

acc_cot = np.mean([c == p for c, p in zip(preds_cot, labels_cot)])
acc_ans = np.mean([c == lab for c, lab in zip(preds_ans, labels_ans)])

In [32]:
preds.shape, labels.shape

(torch.Size([16, 72]), torch.Size([1, 72]))

In [33]:
preds

tensor([[25481,  1666, 13129,  ...,   352, 50256, 50256],
        [25481,   274, 12163,  ...,   513, 50256, 50256],
        [25481,   274,   513,  ...,   718, 50256, 50256],
        ...,
        [25481,   274,   513,  ...,   657, 50256, 50256],
        [25481,   657,   513,  ...,   352, 50256, 50256],
        [25481,    67,   352,  ...,   807, 50256, 50256]])

In [21]:
# tokenizer.decode([segment['labels'][0][0] for segment in segments[1:]])

In [22]:
# import pandas as pd
# res_df = pd.DataFrame(columns=['cpt_path', 'cot', 'acc_cot', 'acc_ans'])

In [83]:
[o.loss for o in cell_outputs]

[None, None, None, None, None, None]

In [ ]:

model.generation_config.pad_token_id = tokenizer.pad_token_id
gen_outputs = [model.generate(inp.reshape(1, -1).to(device), 
                            pad_token_id=tokenizer.eos_token_id,
                            attention_mask=torch.ones_like(inp.reshape(1, -1)).to(device),
                            max_new_tokens=50)[0] for inp in collated['input_ids_generate']]

if args.use_cot:
    gen_outputs = [model.generate(inp.reshape(1, -1).to(device), 
                                    pad_token_id=tokenizer.eos_token_id,
                                    attention_mask=torch.ones_like(inp.reshape(1, -1)).to(device))[0].cpu() for inp in gen_outputs]

labels = collated['labels']
labels_masks = labels > 0

preds_full = [out[len(inp):] for inp, out in zip(collated['input_ids_generate'], gen_outputs)]
# preds_full = [out[len(inp):] for inp, out in zip(collated['input_ids_generate'], gen_outputs_m2)]
labels_full = [lab[m][1:] for lab, m in zip(labels, labels_masks)]

print(len(preds_full), len(labels_full))

preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

preds_cot = [extract_cot(p) for p in preds_full_text]
preds_ans = [extract_answer(p) for p in preds_full_text]

labels_cot = [extract_cot(lab) for lab in labels_full_text]
labels_ans = [extract_answer(lab) for lab in labels_full_text]

acc_cot = np.mean([c == p for c, p in zip(preds_cot, labels_cot)])
acc_ans = np.mean([c == lab for c, lab in zip(preds_ans, labels_ans)])

data = {"inputs": collated['input_ids_generate'],
        "preds_full_text": preds_full_text, "labels_full_text": labels_full_text,
        "preds_cot": preds_cot, "labels_cot": labels_cot,
        "preds_ans": preds_ans, "labels_ans": labels_ans}

print(f"Accuracy COT: {acc_cot}")
print(f"Accuracy Answer: {acc_ans}")

In [77]:
args.use_cot = False

checkpoints = [
    "/workspace-SR006.nfs2/Bulatov_A/rmt/runs/gsm8k/gpt2/SEGM_1x1024_1024_LR3e-04/checkpoint-16500/pytorch_model.bin",
    "/workspace-SR006.nfs2/Bulatov_A/rmt/runs/gsm8k/gpt2/SEGM_1x1024_1024_LR1e-03/checkpoint-24500/pytorch_model.bin",
]

for cpt_path in checkpoints:
    print(cpt_path)
    acc_cot, acc_ans = evaluate_model_on_dataset(cpt_path, valid_dataset)
    res_df.loc[len(res_df)] = [cpt_path.split('/')[-3], args.use_cot, 0, acc_cot]

/workspace-SR006.nfs2/Bulatov_A/rmt/runs/gsm8k/gpt2/SEGM_1x1024_1024_LR3e-04/checkpoint-16500/pytorch_model.bin
500 500
Accuracy COT: 0.172
Accuracy Answer: 1.0
/workspace-SR006.nfs2/Bulatov_A/rmt/runs/gsm8k/gpt2/SEGM_1x1024_1024_LR1e-03/checkpoint-24500/pytorch_model.bin
500 500
Accuracy COT: 0.148
Accuracy Answer: 1.0


In [78]:
res_df

,cpt_path,cot,acc_cot,acc_ans
0,SEGM_1x1024_1024_LR1e-03-cot,True,0.172,0.450
1,SEGM_1x1024_1024_LR3e-04-cot,True,0.156,0.392
2,SEGM_1x1024_1024_LR3e-04,False,0.000,0.172
3,SEGM_1x1024_1024_LR1e-03,False,0.000,0.148


In [60]:
# model.load_state_dict(torch.load(checkpoint_path), strict=False)
model.generation_config.pad_token_id = tokenizer.pad_token_id
model.to(device)
    
collated = collate_fn([sample for sample in valid_dataset])
model.generation_config.pad_token_id = tokenizer.pad_token_id
gen_outputs = [model.generate(inp.reshape(1, -1).to(device), 
                            pad_token_id=tokenizer.eos_token_id,
                            attention_mask=torch.ones_like(inp.reshape(1, -1)).to(device),
                            max_new_tokens=50)[0] for inp in collated['input_ids_generate']]

gen_outputs_m2 = [model.generate(inp.reshape(1, -1).to(device), 
                                    pad_token_id=tokenizer.eos_token_id,
                                    attention_mask=torch.ones_like(inp.reshape(1, -1)).to(device))[0].cpu() for inp in gen_outputs]

labels = collated['labels']
labels_masks = labels > 0

preds_full = [out[len(inp):] for inp, out in zip(collated['input_ids_generate'], gen_outputs_m2)]
labels_full = [lab[m][1:] for lab, m in zip(labels, labels_masks)]

print(len(preds_full), len(labels_full))

preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

preds_cot = [extract_cot(p) for p in preds_full_text]
preds_ans = [extract_answer(p) for p in preds_full_text]

labels_cot = [extract_cot(lab) for lab in labels_full_text]
labels_ans = [extract_answer(lab) for lab in labels_full_text]

acc_cot = np.mean([c == p for c, p in zip(preds_cot, labels_cot)])
acc_ans = np.mean([c == lab for c, lab in zip(preds_ans, labels_ans)])

print(f"Accuracy COT: {acc_cot}")
print(f"Accuracy Answer: {acc_ans}")

500 500
Accuracy COT: 0.148
Accuracy Answer: 1.0


In [ ]:
def extract_cot(text):
        try:
                return text[:text.index(ans_text)]
        except ValueError:
                return ''

def extract_answer(text):
        try:
                return text.split(ans_text)[1]
        except IndexError:
                return ''

In [66]:
preds_full_text

['600<|endoftext|><|endoftext|>',
 '7.5<|endoftext|><|endoftext|>',
 '1400<|endoftext|><|endoftext|>',
 '15<|endoftext|><|endoftext|>',
 '240<|endoftext|><|endoftext|>',
 '20<|endoftext|><|endoftext|>',
 '160<|endoftext|><|endoftext|>',
 '1.5<|endoftext|><|endoftext|>',
 '21.75<|endoftext|><|endoftext|>',
 '21.5<|endoftext|><|endoftext|>',
 '14<|endoftext|><|endoftext|>',
 '88<|endoftext|><|endoftext|>',
 '20<|endoftext|><|endoftext|>',
 '130<|endoftext|><|endoftext|>',
 '16.67<|endoftext|><|endoftext|>',
 '33.33<|endoftext|><|endoftext|>',
 '25<|endoftext|><|endoftext|>',
 '-4<|endoftext|><|endoftext|>',
 '90<|endoftext|><|endoftext|>',
 '46<|endoftext|><|endoftext|>',
 '715<|endoftext|><|endoftext|>',
 '5<|endoftext|><|endoftext|>',
 '10<|endoftext|><|endoftext|>',
 '1<|endoftext|><|endoftext|>',
 '12000<|endoftext|><|endoftext|>',
 '160<|endoftext|><|endoftext|>',
 '5<|endoftext|><|endoftext|>',
 '45<|endoftext|><|endoftext|>',
 '74<|endoftext|><|endoftext|>',
 '460<|endoftext|><|en

In [62]:
preds_cot[:10]

['600', '7.5', '1400', '15', '240', '20', '160', '1.5', '21.75', '21.5']

In [63]:
labels_cot[:10]

['300', '10', '1400', '15', '240', '20', '10', '2', '25', '25']

In [64]:
preds_ans[:10]


['', '', '', '', '', '', '', '', '', '']

In [65]:
labels_ans[:10]

['', '', '', '', '', '', '', '', '', '']

In [13]:

labels = collated['labels'][:, 1:]
labels_masks = labels > 0

preds_full = [out[len(inp):] for inp, out in zip(collated['input_ids_generate'], gen_outputs_m2)]
labels_full = [lab[m][1:] for lab, m in zip(labels, labels_masks)]

print(len(preds_full), len(labels_full))

preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

preds_cot = [extract_cot(p) for p in preds_full_text]
preds_ans = [extract_answer(p) for p in preds_full_text]

labels_cot = [extract_cot(lab) for lab in labels_full_text]
labels_ans = [extract_answer(lab) for lab in labels_full_text]

acc_cot = np.mean([c == p for c, p in zip(preds_cot, labels_cot)])
acc_ans = np.mean([c == lab for c, lab in zip(preds_ans, labels_ans)])

print(f"Accuracy COT: {acc_cot}")
print(f"Accuracy Answer: {acc_ans}")

5 5
Accuracy COT: 0.6
Accuracy Answer: 1.0


In [5]:
dataset = 'booydar/gsm8k'
train_dataset = datasets.load_dataset(dataset, split='train')
valid_dataset = datasets.load_dataset(dataset, split='valid')

# cot
args.use_cot = True

checkpoints = [
    "/workspace-SR006.nfs2/Bulatov_A/rmt/runs/gsm8k/gpt2/SEGM_1x1024_1024_LR1e-03-cot/checkpoint-23500/pytorch_model.bin",
    "/workspace-SR006.nfs2/Bulatov_A/rmt/runs/gsm8k/gpt2/SEGM_1x1024_1024_LR1e-05-cot/checkpoint-5000/pytorch_model.bin",
    "/workspace-SR006.nfs2/Bulatov_A/rmt/runs/gsm8k/gpt2/SEGM_1x1024_1024_LR3e-04-cot/checkpoint-17500/pytorch_model.bin",
]

In [35]:
# cpt_path = "/workspace-SR006.nfs2/Bulatov_A/rmt/runs/test/4_by_4_mult/Llama-3.2-1B-Instruct/smol:qa1-5-1:9/SEGM_1x1024_1024_64_LR3e-04-lora-mnc-distill__short/checkpoint-5000/pytorch_model.bin"
# cpt_path = "/workspace-SR006.nfs2/Bulatov_A/rmt/runs/test/4_by_4_mult/gpt2/SEGM_1x1024_1024_LR3e-04/checkpoint-25000/pytorch_model.bin"
# cpt_path = "/workspace-SR006.nfs2/Bulatov_A/rmt/runs/gsm8k/gpt2/SEGM_1x1024_1024_LR3e-04/checkpoint-16500/pytorch_model.bin"
cpt_path = "/workspace-SR006.nfs2/Bulatov_A/rmt/runs/gsm8k/gpt2/SEGM_1x1024_1024_LR1e-03-cot/checkpoint-23500/pytorch_model.bin"
model.load_state_dict(torch.load(cpt_path), strict=False)

<All keys matched successfully>

In [ ]:
# dataset_dir = "/workspace-SR006.nfs2/Bulatov_A/rmt/data/implicit_chain_of_thought/4_by_4_mult"

# train_path = os.path.join(dataset_dir, "train")
# valid_path = os.path.join(dataset_dir, "valid")
# train_dataset = datasets.load_from_disk(train_path)
# valid_dataset = datasets.load_from_disk(valid_path)



Generating train_no_aug split: 100%|██████████| 6973/6973 [00:00<00:00, 436918.42 examples/s]


In [77]:
class Holder:
    def __init__(self):
        pass
args = Holder()
# args.use_cot = False
args.use_cot = True
args.num_mem_tokens = None

In [118]:

id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
think = ans = tokenizer.bos_token_id
eos = tokenizer.eos_token_id

def collate_fn(batch):
    input_ids, input_ids_generate, labels, labels_mask, attention_mask = [], [], [], [], []
    for sample in batch:
        task, lab, cot = sample['task'], sample['labels'], sample['cot']
        task_tokens = tokenizer.encode(task, add_special_tokens=False)
        labels_tokens = tokenizer.encode(lab, add_special_tokens=False)
        cot_tokens = tokenizer.encode(cot, add_special_tokens=False)

        if args.use_cot:
            full_input = task_tokens + [think] + cot_tokens + [ans] + labels_tokens + [eos]
            gen_input = task_tokens + [think]
        else:
            full_input = task_tokens + [ans] + labels_tokens + [eos]
            gen_input = task_tokens + [ans]
        
        inp_ids = torch.tensor(full_input)
        input_ids.append(inp_ids)
        input_ids_generate.append(torch.tensor(gen_input))


        lab = torch.tensor(full_input)
        lab[:len(task_tokens)] = -100
        labels.append(lab)

        lab_mask = torch.ones_like(inp_ids)
        lab_mask[:len(task_tokens)] = 0
        labels_mask.append(lab_mask)
        attention_mask.append(torch.ones_like(inp_ids))

    input_ids = pad_sequence(input_ids, padding_value=id_pad_value, batch_first=True)
    # input_ids_generate = pad_sequence(input_ids_generate, padding_value=id_pad_value, batch_first=True)
    attention_mask = pad_sequence(attention_mask, padding_value=0, batch_first=True)
    labels = pad_sequence(labels, padding_value=id_pad_value, batch_first=True)
    labels_mask = pad_sequence(labels_mask, padding_value=0, batch_first=True)

    collated = {'input_ids': input_ids,
                'input_ids_generate': input_ids_generate,
                'labels': labels,
                'attention_mask': attention_mask,
                }
    if args.num_mem_tokens is not None:
        # add labels mask only for RMT, ARMT
        collated['labels_mask'] = labels_mask.bool()
    return collated

In [167]:
think_text = tokenizer.decode(think)
ans_text = tokenizer.decode(ans)

def extract_cot(text):
        try:
                return text[:text.index(ans_text)]
        except ValueError:
                return ''

def extract_answer(text):
        try:
                return text.split(ans_text)[1]
        except IndexError:
                return ''
                
def compute_accuracy(eval_pred):
        preds = eval_pred.predictions.argmax(axis=-1)[:, 1:-1]
        labels = eval_pred.label_ids[:, 2:]

        labels_masks = labels > 0
        preds_full = [p[m] for p, m in zip(preds, labels_masks)]
        labels_full = [lab[m] for lab, m in zip(labels, labels_masks)]

        print(len(preds_full), len(labels_full))

        preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
        labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

        preds_cot = [extract_cot(p) for p in preds_full_text]
        preds_ans = [extract_answer(p) for p in preds_full_text]

        labels_cot = [extract_cot(lab) for lab in labels_full_text]
        labels_ans = [extract_answer(lab) for lab in labels_full_text]

        acc_cot = np.mean([c == p for c, p in zip(preds_cot, labels_cot)])
        acc_ans = np.mean([c == lab for c, lab in zip(preds_ans, labels_ans)])

        return {'accuracy_cot': acc_cot, 'accuracy_ans': acc_ans}

In [119]:
batch = [valid_dataset[i] for i in range(10)]
collated = collate_fn(batch)

out = model(**collated)
print(out.keys())

odict_keys(['loss', 'logits', 'past_key_values'])



### no teacher forcing


In [135]:
model.generation_config.pad_token_id = tokenizer.pad_token_id

In [ ]:
model.generation_config.pad_token_id = tokenizer.pad_token_id
gen_outputs = [model.generate(inp.reshape(1, -1), 
                              attention_mask=torch.ones_like(inp.reshape(1, -1)),
                              max_new_tokens=50)[0] for inp in collated['input_ids_generate']]

gen_outputs_m2 = [model.generate(inp.reshape(1, -1), attention_mask=torch.ones_like(inp.reshape(1, -1)))[0] for inp in gen_outputs]

In [233]:
# gen_outputs_m2 = [out[len(inp):] for inp, out in zip(collated['input_ids_generate'], gen_outputs_m2)]

In [234]:
tokenizer.batch_decode(gen_outputs_m2)

['<<4-2=2>> <<2/.5=4>> <<12/4=3>> <<100*3=300>><|endoftext|>300<|endoftext|>',
 '<<1.5*2=3>> <<3+2.5=5.5>> <<1.5+3+5.5=10>><|endoftext|>10<|endoftext|>',
 '<<2000*30/100=600>> <<2000-600=1400>><|endoftext|>1400<|endoftext|>',
 '<<21/7=3>> <<5*3=15>><|endoftext|>15<|endoftext|>',
 '<<200*3=600>> <<600*0.4=240>><|endoftext|>240<|endoftext|>',
 '<<1/2*100=50>> <<3/5*50=30>> <<50+30=80>> <<100-80=20>><|endoftext|>20<|endoftext|>',
 '<<40*2=80>> <<80*2=160>><|endoftext|>160<|endoftext|>',
 '<<12*2=24>> <<4*1=4>> <<4*3=12>> <<24+4+12=40>> <<40/4=10>><|endoftext|>10<|endoftext|>',
 '<<2*2.25=4.50>> <<4.50+3.50+4+3.50=15.00>> <<2*2.50=5.00>> <<15+5+3.50=23.50>><|endoftext|>',
 '<<28/4=7>> <<3*7+1=22>><|endoftext|>22<|endoftext|>']

In [242]:
preds = gen_outputs_m2
labels_masks = labels > 0

preds_full = gen_outputs_m2 #[out[len(inp):] for inp, out in zip(collated['input_ids_generate'], gen_outputs_m2)]
labels_full = [lab[m][1:] for lab, m in zip(labels, labels_masks)]

print(len(preds_full), len(labels_full))

preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

preds_cot = [extract_cot(p) for p in preds_full_text]
preds_ans = [extract_answer(p) for p in preds_full_text]

labels_cot = [extract_cot(lab) for lab in labels_full_text]
labels_ans = [extract_answer(lab) for lab in labels_full_text]

acc_cot = np.mean([c == p for c, p in zip(preds_cot, labels_cot)])
acc_ans = np.mean([c == lab for c, lab in zip(preds_ans, labels_ans)])

print(f"Accuracy COT: {acc_cot}")
print(f"Accuracy Answer: {acc_ans}")

10 10
Accuracy COT: 0.3
Accuracy Answer: 0.6


In [243]:
preds_full_text

['<<4-2=2>> <<2/.5=4>> <<12/4=3>> <<100*3=300>><|endoftext|>300<|endoftext|>',
 '<<1.5*2=3>> <<3+2.5=5.5>> <<1.5+3+5.5=10>><|endoftext|>10<|endoftext|>',
 '<<2000*30/100=600>> <<2000-600=1400>><|endoftext|>1400<|endoftext|>',
 '<<21/7=3>> <<5*3=15>><|endoftext|>15<|endoftext|>',
 '<<200*3=600>> <<600*0.4=240>><|endoftext|>240<|endoftext|>',
 '<<1/2*100=50>> <<3/5*50=30>> <<50+30=80>> <<100-80=20>><|endoftext|>20<|endoftext|>',
 '<<40*2=80>> <<80*2=160>><|endoftext|>160<|endoftext|>',
 '<<12*2=24>> <<4*1=4>> <<4*3=12>> <<24+4+12=40>> <<40/4=10>><|endoftext|>10<|endoftext|>',
 '<<2*2.25=4.50>> <<4.50+3.50+4+3.50=15.00>> <<2*2.50=5.00>> <<15+5+3.50=23.50>><|endoftext|>',
 '<<28/4=7>> <<3*7+1=22>><|endoftext|>22<|endoftext|>']

In [244]:
labels_full_text

['<<4-2=2>> <<2/.5=4>> <<12/4=3>> <<100*3=300>><|endoftext|>300<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><

In [245]:
preds_cot

['<<4-2=2>> <<2/.5=4>> <<12/4=3>> <<100*3=300>>',
 '<<1.5*2=3>> <<3+2.5=5.5>> <<1.5+3+5.5=10>>',
 '<<2000*30/100=600>> <<2000-600=1400>>',
 '<<21/7=3>> <<5*3=15>>',
 '<<200*3=600>> <<600*0.4=240>>',
 '<<1/2*100=50>> <<3/5*50=30>> <<50+30=80>> <<100-80=20>>',
 '<<40*2=80>> <<80*2=160>>',
 '<<12*2=24>> <<4*1=4>> <<4*3=12>> <<24+4+12=40>> <<40/4=10>>',
 '<<2*2.25=4.50>> <<4.50+3.50+4+3.50=15.00>> <<2*2.50=5.00>> <<15+5+3.50=23.50>>',
 '<<28/4=7>> <<3*7+1=22>>']

In [246]:
labels_cot

['<<4-2=2>> <<2/.5=4>> <<12/4=3>> <<100*3=300>>',
 '<<1.5*2=3>> <<3+2.5=5.5>> <<1.5+3+5.5=10>>',
 '<<30/100*2000=600>> <<2000-600=1400>>',
 '<<21/7=3>> <<5*3=15>>',
 '<<200*3=600>> <<600*.4=240>>',
 '<<1/2*100=50>> <<3/5*50=30>> <<50-30=20>>',
 '<<40/2=20>> <<20/2=10>>',
 '<<12*2=24>> <<4*1=4>> <<3*4=12>> <<24+4+12=40>> <<12+4+4=20>> <<40/20=2>>',
 '<<2*2.25=4.50>> <<2*4=8.00>> <<2*2.50=5.00>> <<4.50+8.00+.50+5.00+3.50+3.50=25.00>>',
 '<<32=32>> <<8=8>>']

In [247]:
preds_ans

['300', '10', '1400', '15', '240', '20', '160', '10', '', '22']

In [248]:
labels_ans

['300', '10', '1400', '15', '240', '20', '10', '2', '25', '25']


### teacher forcing


In [193]:
labels_masks.shape, preds.shape

(torch.Size([10, 172]), (10, 173))

In [249]:
# preds = eval_pred.predictions.argmax(axis=-1)[:, :-1]
# labels = eval_pred.label_ids[:, 1:]

preds = out.logits.argmax(dim=-1).cpu().numpy()[:, :-1]
labels = collated['labels'][:, 1:]

labels_masks = labels > 0
# labels_masks[:, :1] = False
preds_full = [p[m][1:] for p, m in zip(preds, labels_masks)]
labels_full = [lab[m][1:] for lab, m in zip(labels, labels_masks)]

print(len(preds_full), len(labels_full))

preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

preds_cot = [extract_cot(p) for p in preds_full_text]
preds_ans = [extract_answer(p) for p in preds_full_text]

labels_cot = [extract_cot(lab) for lab in labels_full_text]
labels_ans = [extract_answer(lab) for lab in labels_full_text]

acc_cot = np.mean([c == p for c, p in zip(preds_cot, labels_cot)])
acc_ans = np.mean([c == lab for c, lab in zip(preds_ans, labels_ans)])

print(f"Accuracy COT: {acc_cot}")
print(f"Accuracy Answer: {acc_ans}")

10 10
Accuracy COT: 0.3
Accuracy Answer: 0.9


In [251]:
preds_full_text

['<<4-2=2>> <<2/.5=4>> <<12/4=3>> <<100*3=300>><|endoftext|>300<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><

In [252]:
labels_full_text

['<<4-2=2>> <<2/.5=4>> <<12/4=3>> <<100*3=300>><|endoftext|>300<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><

In [253]:
preds_cot

['<<4-2=2>> <<2/.5=4>> <<12/4=3>> <<100*3=300>>',
 '<<1.5*2=3>> <<3+2.5=5.5>> <<1.5+3+5.5=10>>',
 '<<2000/100*2000=600>> <<2000-600=1400>>',
 '<<21/7=3>> <<5*3=15>>',
 '<<200*3=600>> <<600*4=240>>',
 '<<1/2*100=50>> <<3/5*50=30>> <<50+30=20>>',
 '<<40*2=20>> <<20/2=10>>',
 '<<12*2=24>> <<4*1=4>> <<4*4=12>> <<24+4+12=40>> <<40+4+12=20>> <<40/20=2>>',
 '<<2*2.25=4.50>> <<4*4=8.00>> <<2*2.50=5.00>> <<4.50+8+00+50+5+00+3.50=2.50=28.00>>',
 '<<28/32>> <<(.8>> <<8']

In [254]:
preds_cot

['<<4-2=2>> <<2/.5=4>> <<12/4=3>> <<100*3=300>>',
 '<<1.5*2=3>> <<3+2.5=5.5>> <<1.5+3+5.5=10>>',
 '<<2000/100*2000=600>> <<2000-600=1400>>',
 '<<21/7=3>> <<5*3=15>>',
 '<<200*3=600>> <<600*4=240>>',
 '<<1/2*100=50>> <<3/5*50=30>> <<50+30=20>>',
 '<<40*2=20>> <<20/2=10>>',
 '<<12*2=24>> <<4*1=4>> <<4*4=12>> <<24+4+12=40>> <<40+4+12=20>> <<40/20=2>>',
 '<<2*2.25=4.50>> <<4*4=8.00>> <<2*2.50=5.00>> <<4.50+8+00+50+5+00+3.50=2.50=28.00>>',
 '<<28/32>> <<(.8>> <<8']

In [255]:
preds_ans

['300', '10', '1400', '15', '240', '20', '10', '2', '25', '']

In [256]:
labels_ans

['300', '10', '1400', '15', '240', '20', '10', '2', '25', '25']

### Older pretokenize

In [ ]:
# from torch.nn.utils.rnn import pad_sequence
# id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
# if args.use_cot in (False, None):
#     inputs_key = 'examples_nocot'
#     labels_key = 'labels_nocot'
# else:
#     inputs_key = 'examples_all'
#     labels_key = 'labels_all'
    
# def collate_fn(batch):
#     input_ids = [torch.tensor(b[inputs_key]) for b in batch]
#     labels = [torch.tensor(b[labels_key]) for b in batch]
#     attention_mask = [torch.ones_like(b, dtype=int) for b in input_ids]
#     # labels_mask defines which input_ids participate in loss calculation
#     labels_mask = [torch.sign(torch.tensor(b[labels_key])) for b in batch]


#     input_ids = pad_sequence(input_ids, padding_value=id_pad_value, batch_first=True)
#     labels = pad_sequence(labels, padding_value=id_pad_value, batch_first=True)
#     attention_mask = pad_sequence(attention_mask, padding_value=0, batch_first=True)
#     labels_mask = pad_sequence(labels_mask, padding_value=0, batch_first=True)

#     collated = {'input_ids': input_ids,
#                 'labels': labels, 
#                 'attention_mask': attention_mask,
#                 }
#     if args.num_mem_tokens is not None:
#         # add labels mask only for RMT, ARMT
#         collated['labels_mask'] = labels_mask.bool()
#     return collated


In [ ]:
# def extract_cot(text):
#     if '<|endoftext|>' not in text:
#         return ''
#     else:
#         return text.split('<|endoftext|>')[0].strip()

# def extract_answer(text):
#     if '####' not in text:
#         return ''
#     else:
#         ans = text.split('####')[-1]
#         ans = ans.split('<|endoftext|>')[0]
#         return ans.strip()
        
# def compute_accuracy(eval_pred):
#     preds = eval_pred.predictions[:, :-1]
#     labels = eval_pred.label_ids[:, 1:]
#     # inputs = eval_pred.inputs
#     # losses = eval_pred.losses

#     # labels = collated['labels'][:, 1:]

#     labels_masks = labels > 0
#     preds_full = [p[m] for p, m in zip(preds, labels_masks)]
#     labels_full = [l[m] for l, m in zip(labels, labels_masks)]

#     preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
#     labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

#     preds_cot = [extract_cot(p) for p in preds_full_text]
#     preds_ans = [extract_answer(p) for p in preds_full_text]

#     labels_cot = [extract_cot(l) for l in labels_full_text]
#     labels_ans = [extract_answer(l) for l in labels_full_text]
    
#     # Calculate accuracy only on the unignored tokens
#     acc_cot = np.mean([c == l for c, l in zip(preds_cot, labels_cot)])
#     acc_ans = np.mean([c == l for c, l in zip(preds_ans, labels_ans)])

#     return {'accuracy_cot': acc_cot, 'accuracy_ans': acc_ans}

In [39]:
# predictions = eval_pred.predictions
# label_ids = eval_pred.label_ids
# inputs = eval_pred.inputs
# losses = eval_pred.losses
# # elements = (self.predictions, self.label_ids)

In [40]:
batch = [valid_dataset[i] for i in range(10)]
collated = collate_fn(batch)

out = model(**collated)
print(out.keys())

odict_keys(['loss', 'logits', 'past_key_values'])


In [41]:
out.loss

tensor(0.0672, grad_fn=<NllLossBackward0>)

In [42]:
preds = out.logits.argmax(dim=-1).cpu().numpy()
preds.shape

(10, 175)

In [43]:
preds_text = tokenizer.batch_decode(preds, add_special_tokens=True)

In [44]:
# preds_text[0].split('<|endoftext|>')

In [46]:
# ''.split('<|endoftext|>')[1]

In [47]:
labels = collated['labels'][:, 1:]

labels_masks = labels > 0
preds_full = [p[m] for p, m in zip(preds[:, :-1], labels_masks)]
labels_full = [l[m] for l, m in zip(labels, labels_masks)]

preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

preds_cot = [extract_cot(p) for p in preds_full_text]
preds_ans = [extract_answer(p) for p in preds_full_text]

labels_cot = [extract_cot(l) for l in labels_full_text]
labels_ans = [extract_answer(l) for l in labels_full_text]

In [48]:
# labels = collated['labels'][:, 1:]

# labels_masks = labels > 0
# preds_full = [p[m] for p, m in zip(preds[:, :-1], labels_masks)]
# labels_full = [l[m] for l, m in zip(labels, labels_masks)]

# preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
# labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

# preds_cot = [p.split('<|endoftext|>')[0].strip() for p in preds_full_text]
# labels_cot = [l.split('<|endoftext|>')[0].strip() for l in labels_full_text]

# # preds_ans = [p.split('<|endoftext|>')[1].strip()[4:] for p in preds_full_text]
# # labels_ans = [l.split('<|endoftext|>')[1].strip()[4:] for l in labels_full_text]

# preds_ans = [p.split('####')[1].strip()[4:] for p in preds_full_text]
# labels_ans = [l.split('####')[1].split('astrip()[4:] for l in labels_full_text]



In [49]:
tokenizer.batch_decode(collated['input_ids'])

['John cuts his grass to 2 inches.  It grows .5 inches per month.  When it gets to 4 inches he cuts it back down to 2 inches.  It cost $100 to get his grass cut.  How much does he pay per year?<|endoftext|><<4-2=2>> <<2/.5=4>> <<12/4=3>> <<100*3=300>><|endoftext|>300<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|end

In [50]:
labels_full_text

['<|endoftext|><<4-2=2>> <<2/.5=4>> <<12/4=3>> <<100*3=300>><|endoftext|>300<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><

In [51]:
preds_cot

['', '', '', '', '', '', '', '', '', '']

In [52]:
labels_cot

['', '', '', '', '', '', '', '', '', '']

In [53]:
labels_ans

['', '', '', '', '', '', '', '', '', '']

In [54]:
preds_ans

['', '', '', '', '', '', '', '', '', '']

In [27]:
labels_text
[0].split('<|endoftext|>')

NameError: name 'labels_text' is not defined

In [28]:
for p, l, m in zip(preds, collated['labels'], collated['labels_mask']):
    print(p[m], l[m])

KeyError: 'labels_mask'

In [42]:
preds[0]

array([   11,   860,   860,   604,  1635,   657,   657,   767,   657,
         642,   642,   642,   642,   718,   352,  1343,   657,   657,
         718,   604,   860,   657,   357,   642,   642,   352,   352,
         352,   352,  1267,  1343,   657,   657,   642,   860,   657,
         767,   657,   357,   642,   642,   718,   657,   362,   807,
         657,  1267,  1343,   657,   657,   657,   657,   718,   604,
         860,   657,   220, 50256,  1303, 21017,   642,   642,   718,
         657,   807,   362,   657,   352,   220, 50256,  1303])

In [27]:
pred_texts = tokenizer.batch_decode(preds, skip_special_tokens=False)
print(pred_texts)

[', 9 9 4 * 0 0 7 0 5 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0 <|endoftext|> #### 5 5 6 0 8 2 0 1 <|endoftext|> #', ' the 0 1 6 0 1 0 8 3 6 6 7 1 1 3 + 0 4 8 7 0 2 ( 6 1 0 9 3 2 ) + 0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 ) + 0 0 0 2 7 3 6 3 <|endoftext|> #### 6 1 4 9 8 6 8 3 <|endoftext|> #', ' the 0 8 4 + 2 8 3 0 8 8 0 0 5 7 + 0 4 8 3 4 8 ( 8 4 8 8 1 9 ) + 0 0 6 7 3 9 0 ( 8 4 4 6 5 8 1 ) + 0 0 0 2 3 6 5 6 <|endoftext|> #### 8 4 4 8 8 4 7 6 <|endoftext|> #', ', 2 3 2\n 0 0 0 1 9 9 0 9 2 1 + 0 2 1 2 7 1 ( 9 2 0 5 8 1 ) + 0 0 8 1 8 5 2 ( 9 2 8 6 6 7 2 ) + 0 0 0 5 1 5 1 2 <|endoftext|> #### 9 2 8 1 8 2 4 2 <|endoftext|> #', ' the 1 - 0 0 0 1 3 2 0 0 4 6 8 5 + 0 0 5 6 6 3 ( 0 4 1 5 2 4 ) + 0 0 0 8 9 3 4 ( 0 4 1 3 2 8 4 ) + 0 0 0 0 5 6 6 3 <|endoftext|> #### 0 4 1 3 7 4 1 4 <|endoftext|> #', ', 4 5 1 0 8 2 8 1 2 2 5 2 4 2 + 0 9 8 1 8 1 ( 2 4 1 6 0 2 ) + 0 0 4 0 5 8 4 ( 2 4 5 6 5 0 5 ) + 0 0 0 1 4 4 2 4 <|endoftext|> #### 2 4 5 7 9 4 7 4 <|endoftext|

In [28]:
labels_texts = tokenizer.batch_decode(collated['input_ids'], skip_special_tokens=False)
print(labels_texts)

[' 5 6 3 2 * 7 4 3 4 <|endoftext|> 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0 <|endoftext|> #### 5 5 6 0 8 2 0 1 <|endoftext|>', ' 6 9 1 5 * 6 4 4 7 <|endoftext|> 6 7 1 1 3 + 0 4 8 7 0 2 ( 6 1 0 9 3 2 ) + 0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 ) + 0 0 0 2 7 3 6 3 <|endoftext|> #### 6 1 4 9 8 6 8 3 <|endoftext|>', ' 6 7 3 9 * 8 9 1 7 <|endoftext|> 8 0 0 5 7 + 0 4 8 3 4 8 ( 8 4 8 8 1 9 ) + 0 0 6 7 3 9 0 ( 8 4 4 6 5 8 1 ) + 0 0 0 2 3 6 5 6 <|endoftext|> #### 8 4 4 8 8 4 7 6 <|endoftext|>', ' 3 0 3 4 * 3 4 6 5 <|endoftext|> 9 0 9 2 1 + 0 2 1 2 7 1 ( 9 2 0 5 8 1 ) + 0 0 8 1 8 5 2 ( 9 2 8 6 6 7 2 ) + 0 0 0 5 1 5 1 2 <|endoftext|> #### 9 2 8 1 8 2 4 2 <|endoftext|>', ' 0 3 3 7 * 8 5 6 5 <|endoftext|> 0 4 6 8 5 + 0 0 5 6 6 3 ( 0 4 1 5 2 4 ) + 0 0 0 8 9 3 4 ( 0 4 1 3 2 8 4 ) + 0 0 0 0 5 6 6 3 <|endoftext|> #### 0 4 1 3 7 4 1 4 <|endoftext|>', ' 3 6 0 6 * 4 3 8 7 <|endoftext|> 2 5 2 4 2 + 0 9 8 1 8 1 ( 2 4 1 6 0 2 ) + 0 0 4 0 5 8 4 ( 2 4 5 6 5 0 5 ) + 0 0 

In [29]:
collated['input_ids'].shape

torch.Size([10, 71])

In [30]:
pred_texts[0]

', 9 9 4 * 0 0 7 0 5 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0 <|endoftext|> #### 5 5 6 0 8 2 0 1 <|endoftext|> #'

In [33]:
preds[0]

array([   11,   860,   860,   604,  1635,   657,   657,   767,   657,
         642,   642,   642,   642,   718,   352,  1343,   657,   657,
         718,   604,   860,   657,   357,   642,   642,   352,   352,
         352,   352,  1267,  1343,   657,   657,   642,   860,   657,
         767,   657,   357,   642,   642,   718,   657,   362,   807,
         657,  1267,  1343,   657,   657,   657,   657,   718,   604,
         860,   657,   220, 50256,  1303, 21017,   642,   642,   718,
         657,   807,   362,   657,   352,   220, 50256,  1303])

In [ ]:
collated['input_ids'][0]

torch.Size([71])

In [38]:
for p, t in zip(preds[0], collated['input_ids'][0][1:]):
    # print(p, tokenizer.decode([p]), t, tokenizer.decode([t]))
    print(tokenizer.decode([p]), tokenizer.decode([t]))

,  6
 9  3
 9  2
 4  *
 *  7
 0  4
 0  3
 7  4
 0  
 5 <|endoftext|>
 5  5
 5  5
 5  5
 6  6
 1  1
 +  +
 0  0
 0  0
 6  6
 4  4
 9  9
 0  0
 (  (
 5  5
 5  5
 1  1
 1  1
 1  1
 1  1
 )  )
 +  +
 0  0
 0  0
 5  5
 9  9
 0  0
 7  7
 0  0
 (  (
 5  5
 5  5
 6  6
 0  0
 2  2
 8  8
 0  0
 )  )
 +  +
 0  0
 0  0
 0  0
 0  0
 6  6
 4  4
 9  9
 0  0
   
<|endoftext|> <|endoftext|>
 #  #
### ###
 5  5
 5  5
 6  6
 0  0
 8  8
 2  2
 0  0
 1  1
   
<|endoftext|> <|endoftext|>
